# Time Series Analysis: "The Final Project"

`End? No, the journey doesn't end here. Death is just another path. One that we all must take.
-J.R.R. Tolkien, The Return of the King`

LT8: Carpio, Fiesta, Navarro, Nigam

---

## Libraries

In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from IPython.display import display, HTML
from tsa_functions import *

np.set_printoptions(precision=4)
pd.set_option('precision', 4)
# pd.set_option("max_rows", None)
# pd.reset_option('^display.', silent=True)

import statsmodels.api as sm
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.statespace.exponential_smoothing import ExponentialSmoothing


def rmsse(y_true, y_pred, ts):
    score = np.sqrt(np.mean((y_true - y_pred)**2)/np.mean((ts[1:] - ts[:-1])**2))
    return score
    
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings('ignore')

---

## M5 Forecasting

For this "final project", we will be forecasting the <b><u>level 9</b></u> series (unit sales of all products, aggregated for each store and department).

Load `sales_train_evaluation.csv` and use observations from `d_1 to d_1913` for training and `d_1914 to d_1941` for testing.

In [2]:
# Pre-processing code here
df_calendar = pd.read_csv('calendar.csv')
df_price = pd.read_csv('sell_prices.csv')
df_sales = pd.read_csv('sales_train_validation.csv')
df_sales_2 = pd.read_csv('sales_train_evaluation.csv')

In [3]:
date_list = [d.strftime('%Y-%m-%d') for d in pd.date_range(start='2011-01-29', end='2016-04-24', freq='D')]
df_sales.columns = list(df_sales.columns[:6])+date_list
df_sales

,id,item_id,dept_id,cat_id,store_id,state_id,2011-01-29,2011-01-30,2011-01-31,2011-02-01,...,2016-04-15,2016-04-16,2016-04-17,2016-04-18,2016-04-19,2016-04-20,2016-04-21,2016-04-22,2016-04-23,2016-04-24
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [4]:
df_sales_store_dept = df_sales.groupby(['store_id', 'dept_id']).sum()
df_sales_store_dept[df_sales_store_dept<10] = np.nan
df_sales_store_dept.columns = pd.date_range(start='2011-01-29', end='2016-04-24', freq='D')
df_sales_store_dept = df_sales_store_dept.T.apply(lambda x: x.interpolate(method='time')).T
df_sales_store_dept.fillna(method="bfill", inplace=True)
df_sales_store_dept

2011-01-29  2011-01-30  2011-01-31  2011-02-01  \
store_id dept_id                                                       
CA_1     FOODS_1           297.0       284.0       214.0       175.0   
         FOODS_2           674.0       655.0       396.0       476.0   
         FOODS_3          2268.0      2198.0      1398.0      1607.0   
         HOBBIES_1         528.0       489.0       409.0       383.0   
         HOBBIES_2          28.0        25.2        22.4        19.6   
...                          ...         ...         ...         ...   
WI_3     FOODS_3          2293.0      2383.0      1841.0      1965.0   
         HOBBIES_1         256.0       342.0       228.0       183.0   
         HOBBIES_2          22.0        14.0        20.0        11.0   
         HOUSEHOLD_1       584.0       541.0       420.0       327.0   
         HOUSEHOLD_2       148.0       195.0       106.0        94.0   

                      2011-02-02  2011-02-03  2011-02-04  2011-02-05  \
store_id dept_id                                                       
CA_1     FOODS_1           182.0       191.0       224.0       263.0   
         FOODS_2           354.0       486.0       554.0       747.0   
         FOODS_3          1496.0      1730.0      1915.0      2952.0   
         HOBBIES_1         263.0       453.0       339.0       750.0   
         HOBBIES_2          16.8        14.0        16.5        19.0   
...                          ...         ...         ...         ...   
WI_3     FOODS_3          1427.0      2760.0      2727.0      3786.0   
         HOBBIES_1          70.0       285.0       357.0       399.0   
         HOBBIES_2          11.5        12.0        12.5        13.0   
         HOUSEHOLD_1       151.0       392.0       492.0       647.0   
         HOUSEHOLD_2        53.0       122.0       159.0       186.0   

                      2011-02-06  2011-02-07  ...  2016-04-15  2016-04-16  \
store_id dept_id                              ...                           
CA_1     FOODS_1        245.0000    176.0000  ...       293.0       317.0   
         FOODS_2        581.0000    559.0000  ...       435.0       499.0   
         FOODS_3       2592.0000   1711.0000  ...      1978.0      2665.0   
         HOBBIES_1      425.0000    375.0000  ...       370.0       690.0   
         HOBBIES_2       16.3333     13.6667  ...        31.0        32.0   
...                          ...         ...  ...         ...         ...   
WI_3     FOODS_3       3179.0000   1924.0000  ...      2534.0      2659.0   
         HOBBIES_1      216.0000    222.0000  ...       304.0       376.0   
         HOBBIES_2       17.0000     21.0000  ...        44.0        35.0   
         HOUSEHOLD_1    418.0000    354.0000  ...       676.0       708.0   
         HOUSEHOLD_2    119.0000    108.0000  ...       169.0       179.0   

                      2016-04-17  2016-04-18  2016-04-19  2016-04-20  \
store_id dept_id                                                       
CA_1     FOODS_1           268.0       302.0       237.0       287.0   
         FOODS_2           637.0       529.0       391.0       423.0   
         FOODS_3          3099.0      2055.0      1860.0      1780.0   
         HOBBIES_1         597.0       517.0       461.0       333.0   
         HOBBIES_2          58.0        25.0        34.0        30.0   
...                          ...         ...         ...         ...   
WI_3     FOODS_3          2449.0      1991.0      1715.0      1579.0   
         HOBBIES_1         200.0       149.0       249.0       202.0   
         HOBBIES_2          51.0        25.0        27.0        42.0   
         HOUSEHOLD_1       802.0       609.0       558.0       535.0   
         HOUSEHOLD_2       201.0       161.0       120.0       148.0   

                      2016-04-21  2016-04-22  2016-04-23  2016-04-24  
store_id dept_id                                                      
CA_1     FOODS_1           297.0       331.0       419.0       329.

In [5]:
df_sales_store_2 = df_sales_2.groupby(['store_id']).sum()
df_sales_store_dept_2 = df_sales_2.groupby(['store_id', 'dept_id']).sum()
truth = df_sales_store_dept_2.iloc[:, -28:].T
truth

store_id    CA_1                                                              \
dept_id  FOODS_1 FOODS_2 FOODS_3 HOBBIES_1 HOBBIES_2 HOUSEHOLD_1 HOUSEHOLD_2   
d_1914       294     458    2141       546        43         780         210   
d_1915       259     500    1759       395        31         583         176   
d_1916       268     412    1787       418        33         615         182   
d_1917       328     400    1718       385        47         552         188   
d_1918       387     463    2182       550        34         708         249   
d_1919       455     653    2842       574        62         991         321   
d_1920       403     720    3131       625        59        1118         341   
d_1921       262     620    2396       478        51         763         224   
d_1922       332     525    2140       582        46         761         208   
d_1923       272     531    2037       522        43         637         192   
d_1924       311     450    1974       405        34         578         166   
d_1925       424     560    2319       529        41         702         229   
d_1926       474     683    2949       579        67        1048         269   
d_1927       320     539    2771       451        62         768         301   
d_1928       298     617    2330       592        49         752         221   
d_1929       326     627    1943       334        55         623         196   
d_1930       279     446    1962       453        52         593         197   
d_1931       256     457    1950       546        50         597         193   
d_1932       412     545    2408       546        42         777         221   
d_1933       435     611    3000       732        74        1055         338   
d_1934       415     781    3521       631        56         965         338   
d_1935       277     555    2366       389        28         725         228   
d_1936       302     476    2016       347        46         607         155   
d_1937       397     427    1992       353        28         602         196   
d_1938       330     411    2113       354        52         658         218   
d_1939       310     434    2249       494        45         665         236   
d_1940       427     627    2729       656        63         978         284   
d_1941       334     719    3274       588        58        1022         294   

store_id    CA_2                  ...      WI_2                          \
dept_id  FOODS_1 FOODS_2 FOODS_3  ... HOBBIES_2 HOUSEHOLD_1 HOUSEHOLD_2   
d_1914       439     444    1771  ...        28         874         110   
d_1915       362     436    1508  ...        27         810         148   
d_1916       445     359    1420  ...        38         705         124   
d_1917       445     376    1636  ...        25         920         139   
d_1918       478     421    1962  ...        22        1635         162   
d_1919       711     597    2733  ...        55        1263         152   
d_1920       576     758    2683  ...        31        1133         156   
d_1921       475     625    1895  ...        30         981         123   
d_1922       476     459    1689  ...        23        1267         143   
d_1923       537     557    1921  ...        35         872         146   
d_1924       557     433    1659  ...        83         865         140   
d_1925       494     556    1935  ...        38         935         138   
d_1926       719     802    2776  ...        26        1133         183   
d_1927       607     629    2222  ...        25         730         133   
d_1928       404     607    1719  ...        15         888         144   
d_1929       435     490    1755  ...        23         795         127   
d_1930       386     439    1681  ...        31         989         141   
d_1931       494     413    1584  ...        51         809         115   
d_1932       683     510    2062  ...        51        1008         154   
d_1933       628     621

---

## Part 1. Baseline Methods (10 pts.)

### Q1. (10 pts.)

Extract all level 9 series from the dataset.

For each series, generate a 28-step forecast using the methods enumerated below and calculate the `RMSSE` against the test set:

1. `Naive`


2. `Seasonal Naive`


3. `SES`


4. `Holt's Linear`


5. `Additive Holt-Winters`

Summarize the metrics in a dataframe and print it.

In [6]:
train_eval = pd.read_csv('sales_train_evaluation.csv')

train_eval_l9 = train_eval.groupby(['store_id', 'dept_id']).sum()
train = train_eval_l9.iloc[:, :-28]
test = train_eval_l9.iloc[:, -28:]

In [7]:
def baseline(data, test, key, h):
    df_train = data.loc[key].astype(float)
    df_train = df_train.apply(lambda x: np.nan if x < 10 else x)
    df_train.index = pd.date_range(start='2011-01-29', end='2016-04-24',
                                   freq='D')
    df_train = df_train.interpolate(method='time')
    df_train = df_train.fillna(method='bfill')
    
    df_test = test.loc[key].ravel()
    
    naive = naivef(df_train, h)
    snaive = snaivef(df_train, h, 7)
    
    SES_model = SimpleExpSmoothing(df_train).fit()
    ses = SES_model.forecast(h).to_numpy()
    
    HL = ExponentialSmoothing(df_train, trend=True).fit(optimized=True)
    holt_linear = HL.forecast(h)
    
    AHW = ExponentialSmoothing(df_train, trend=True, seasonal=7
                               ).fit(optimized=True, maxiter=100)
    hw_add = AHW.forecast(h).to_numpy()
    
    df_naive = rmsse(df_test, naive, df_train.values)
    df_snaive = rmsse(df_test, snaive, df_train.values)
    df_ses = rmsse(df_test, ses, df_train.values)
    df_hl = rmsse(df_test, holt_linear, df_train.values)
    df_ahw = rmsse(df_test, hw_add, df_train.values)
    
    df_pred = pd.DataFrame([naive, snaive, ses, holt_linear, hw_add],
                           index=['Naive', 'S. Naive', 'SES',
                                   "Holt's Linear", 'Additive Holt-Winters'])
    df_pred.to_csv(f'baseline_predictions/pred_{key}.csv')
    
    df_acc = pd.DataFrame([[df_naive, df_snaive, df_ses, df_hl, df_ahw]],
                          columns=['Naive', 'S. Naive', 'SES',
                                   "Holt's Linear", 'Additive Holt-Winters'],
                          index=[key])
    return df_acc

In [8]:
h = 28

df_acc = pd.DataFrame()
for key in train.index:
    temp = baseline(train, test, key, h)
    df_acc = df_acc.append(temp)
    
df_acc.to_csv('baseline_accuracies.csv')

In [9]:
baselines = pd.read_csv('baseline_accuracies.csv', index_col=0)
baselines

,Naive,S. Naive,SES,Holt's Linear,Additive Holt-Winters
"('CA_1', 'FOODS_1')",0.9319,0.7312,0.9180,0.9171,0.7079
"('CA_1', 'FOODS_2')",2.0535,0.8269,2.0535,2.0589,0.5624
"('CA_1', 'FOODS_3')",1.7113,0.4944,1.7084,1.0763,0.4469
"('CA_1', 'HOBBIES_1')",1.4583,0.7619,0.8820,0.8807,0.6397
"('CA_1', 'HOBBIES_2')",1.9340,1.1429,0.8831,0.8747,0.7199
...,...,...,...,...,...
"('WI_3', 'FOODS_3')",1.0986,0.7701,1.1103,1.1094,0.8050
"('WI_3', 'HOBBIES_1')",0.9587,0.9150,0.9277,0.9251,0.7908
"('WI_3', 'HOBBIES_2')",1.5458,1.0980,0.9523,0.9465,0.9388
"('WI_3', 'HOUSEHOLD_1')",1.4203,0.6588,1.4212,1.0809,0.6561


---

## Part 2. LightGBM (30 pts.)

### Q2. (10 pts.)

For all series, use an un-tuned `LightGBM` with 56-day lookback that uses a one-step recursive forecasting strategy to generate a 28-step forecast.

Calculate the `RMSSE` against the test set, then summarize the metrics in a dataframe and print it.

In [10]:
def xy(data, lb, h):
    x = []
    y = []
    
    for i in range(data.shape[0]):
        if i+lb+h > data.shape[0]:
            break
        x.append(data.iloc[i : i+lb].values)
        y.append(data.iloc[i+lb : i+lb+h].values)
    
    return np.asarray(x), np.asarray(y)

In [11]:
# Your code here
w = 56
h = 1
test_size = 28

def onestep(col):
    train_x, train_y = xy(df_sales_store_dept.loc[col], w, h)
    model = lgb.LGBMRegressor(random_state=1).fit(train_x, train_y.ravel()) 

    forecast_recursive = []
    x = train_x[:1, -w:].copy() # Must copy to avoid memory warnings

    for i in range(test_size):
        y_hat = model.predict(x)
        x = np.append(x[:, 1:], y_hat).reshape(1, -1)
        forecast_recursive.append(y_hat) 

    forecast_recursive = np.concatenate(forecast_recursive).ravel()
    
    return forecast_recursive

In [12]:
res = []
df_pred = pd.DataFrame()

for col in df_sales_store_dept.T.columns:
    forecast_recursive = onestep(col)
    df_pred[col] = forecast_recursive

    error = rmsse(truth[col].to_numpy(), forecast_recursive,
                  df_sales_store_dept.loc[col].to_numpy())
    res.append(error)
    
df_pred.index = truth.index

In [13]:
df_res = pd.DataFrame({'rmsse':res})
df_res.index = df_sales_store_dept.index

df_res.to_csv('onesteplgb_res.csv')
df_pred.to_csv('onesteplgb_pred.csv')

In [14]:
onesteplgb_res = pd.read_csv('onesteplgb_res.csv')
onesteplgb_res

,store_id,dept_id,rmsse
0,CA_1,FOODS_1,2.3616
1,CA_1,FOODS_2,1.6204
2,CA_1,FOODS_3,2.2575
3,CA_1,HOBBIES_1,1.3143
4,CA_1,HOBBIES_2,1.9710
...,...,...,...
65,WI_3,FOODS_3,1.5365
66,WI_3,HOBBIES_1,1.2885
67,WI_3,HOBBIES_2,1.6089
68,WI_3,HOUSEHOLD_1,2.2554


### Q3. (10 pts.)

For all series, use an un-tuned `LightGBM` with 56-day lookback that uses a direct forecasting strategy to generate a 28-step forecast.

Calculate the `RMSSE` against the test set, then summarize the metrics in a dataframe and print it.

In [15]:
def train_test_split(df_data, w, h, test_size, features):

    ### Train-Test split from DataFrame ###
    
    ### Exogenous Variables ###
    feat_len = len(features)
    df_series = df_data[features].to_frame()
    
    split_time = df_series.shape[0] - test_size
    
    df_train = df_series.iloc[:split_time, :]
    df_test = df_series.iloc[split_time - w:, :]

    return df_train, df_test

def create_xy(series, label, w, h, shuffle=False):
    
    ### Create (X,Y) from DataFrame ####
    
    x = []
    y = []
    for i in range(series.shape[0]):
        if series.iloc[(i + w):(i + w + h), :].shape[0] < h:
            break
        x.append(series.iloc[i:(i + w), :])
        y.append(series.iloc[(i + w):(i + w + h), :][label])
    x = np.dstack(x)
    y = np.dstack(y)
    
    # Reshape x to (samples, features, lookback)
    x = np.swapaxes(x, 0, 2)
    
    # Reshape x to (samples, horizon)
    y = np.swapaxes(y, 0, 2)
    y = y[:, :, 0]
    
    return x,y

In [16]:
train = df_sales_store_dept.transpose()
test = truth.copy()

In [17]:
from sklearn.multioutput import MultiOutputRegressor

w = 56
h = 28
test_size = 28
    
def lgbm_direct(series):   
    target = series
    features = series

    df_train, df_test = train_test_split(train, w, h, test_size, features)

    train_x, train_y = create_xy(df_train, target, w, h, shuffle=False)
    test_x, test_y = create_xy(df_test, target, w, h, shuffle=False)

    train_x = train_x.reshape((train_x.shape[0], train_x.shape[1]*train_x.shape[2]))
    test_x = test_x.reshape((test_x.shape[0], test_x.shape[1]*test_x.shape[2]))
    
    model = MultiOutputRegressor(lgb.LGBMRegressor(), n_jobs = -1)
    model.fit(train_x, train_y)
    forecast_direct = model.predict(test_x).ravel()
    rmsse_val = rmsse(test[series].values, forecast_direct, train[series].values)
    
    return rmsse_val, forecast_direct

In [18]:
series_list = []
rmsse_list = []
forecasts_list = []
for series in train.columns:
    series_list.append(str(series))
    rmsse_list.append(lgbm_direct(series)[0])
    forecasts_list.append(lgbm_direct(series)[1])

In [19]:
df_acc = pd.DataFrame()
df_acc['Series']=series_list
df_acc['RMSSE']=rmsse_list

df_pred = pd.DataFrame()
for col, val in zip(series_list, forecasts_list):
    df_pred[col] = list(val)

In [20]:
df_acc.to_csv('direct_lgb_acc.csv')
df_pred.to_csv('direct_lgb_predictions.csv')

In [21]:
direct_lgb_acc = pd.read_csv('direct_lgb_acc.csv', index_col=0)
direct_lgb_acc

,Series,RMSSE
0,"('CA_1', 'FOODS_1')",0.6209
1,"('CA_1', 'FOODS_2')",0.7232
2,"('CA_1', 'FOODS_3')",0.6356
3,"('CA_1', 'HOBBIES_1')",0.6946
4,"('CA_1', 'HOBBIES_2')",0.7450
...,...,...
65,"('WI_3', 'FOODS_3')",0.5974
66,"('WI_3', 'HOBBIES_1')",0.9010
67,"('WI_3', 'HOBBIES_2')",1.1149
68,"('WI_3', 'HOUSEHOLD_1')",0.8238


### Q4. (10 pts.)

For all series, generate a 28-step forecast by combining the forecasts generated by the models in Q2 and Q3 (i.e. simple averaging).

Calculate the `RMSSE` against the test set, then summarize the metrics in a dataframe and print it.

In [22]:
df_pred_one = pd.read_csv('onesteplgb_pred.csv', index_col=0)
df_pred_direct = pd.read_csv('direct_lgb_predictions.csv', index_col=0)

avg_pred = (df_pred_direct + df_pred_one)/2
avg_pred.columns = truth.columns
avg_pred

store_id      CA_1                                                       \
dept_id    FOODS_1   FOODS_2    FOODS_3 HOBBIES_1 HOBBIES_2 HOUSEHOLD_1   
0         261.2818  437.2776  1879.7423  522.0911   41.2386    573.9494   
1         283.9435  461.4041  1850.2720  466.9726   28.3185    530.6807   
2         214.4667  377.1371  1566.9691  485.3161   30.0054    437.6610   
3         251.2545  361.7239  1435.1100  385.9711   30.5065    424.8404   
4         257.5164  368.4810  1638.2907  408.3965   34.2531    473.4849   
5         293.5153  507.6179  1941.5575  472.4501   34.3219    628.8661   
6         293.1014  576.1388  2197.2244  466.5276   35.2929    685.8293   
7         277.1789  545.4917  2003.6161  508.0068   31.9796    605.7920   
8         281.1288  553.7264  1905.8136  505.4924   31.6972    535.2115   
9         257.1594  501.4633  1786.4162  480.1175   26.0005    457.7093   
10        237.1017  435.1257  1619.4800  386.0493   29.9704    445.0611   
11        295.6478  439.0508  1708.3562  412.7527   30.3903    485.1863   
12        327.6878  549.6666  2095.3801  508.3327   39.2988    644.3180   
13        281.8054  573.4486  2208.7990  501.0849   34.9693    648.6044   
14        273.4993  551.4779  2048.0643  499.7360   37.5136    586.9925   
15        258.1450  540.2248  1943.7808  492.4266   36.8646    506.8646   
16        238.9344  462.6420  1571.3548  484.2333   31.4409    447.3326   
17        227.2848  365.6931  1488.4571  426.8255   34.1896    431.1104   
18        268.6245  387.5511  1622.0077  384.1843   31.4812    493.9926   
19        320.3661  467.7985  1928.0688  532.4788   38.2131    637.2330   
20        302.7936  502.3710  2032.7071  460.2439   33.3991    657.2298   
21        278.7487  509.0115  1948.6120  481.0210   34.9300    570.6045   
22        255.1912  483.5060  1894.1485  417.6934   33.7343    515.2458   
23        205.2019  401.6657  1539.3068  425.5206   28.9572    463.7503   
24        217.3255  353.8401  1459.6938  358.5337   28.5945    443.3166   
25        269.4583  408.5244  1646.0574  383.0864   34.1214    533.0033   
26        314.4790  466.8235  1915.6381  436.8535   35.2717    637.8631   
27        288.3809  484.2457  2157.8469  436.6311   39.1779    699.2364   

store_id                  CA_2                       ...      WI_2  \
dept_id  HOUSEHOLD_2   FOODS_1   FOODS_2    FOODS_3  ... HOBBIES_2   
0           184.9458  362.0290  303.3447  1436.1476  ...   19.3377   
1           203.6544  379.4819  302.3438  1427.2631  ...   22.4009   
2           143.4191  289.6254  275.4567  1165.9110  ...   23.7750   
3           153.9564  310.9161  257.6905  1257.8601  ...   21.1209   
4           159.6484  372.5547  258.3005  1306.2972  ...   17.0042   
5           200.0234  399.9851  378.7659  1558.7864  ...   21.3249   
6           199.2323  419.6552  418.5005  1656.1076  ...   21.9732   
7           185.2743  369.7499  320.2438  1499.0103  ...   20.9820   
8           185.0225  356.7469  318.2721  1467.9238  ...   19.5196   
9           156.9146  326.6680  297.1047  1214.8717  ...   25.4819   
10          142.9547  342.6131  262.4302  1202.6577  ...   22.6325   
11          159.9951  381.6061  293.3446  1474.3573  ...   21.0884   
12          200.1658  422.9585  399.6361  1597.4056  ...   24.9097   
13          203.9550  397.4366  403.8913  1646.4020  ...   20.9894   
14          191.7621  398.8756  301.4770  1348.9063  ...   22.8652   
15          191.1952  387.8938  320.8134  1538.2136  ...   22.6372   
16          155.8628  334.5454  319.3849  1201.8392  ...   24.3781   
17          148.5298  320.4897  275.3584  1131.7918  ...   17.8273   
18          151.8465  374.3890  268.0884  1391.6985  ...   26.6092   
19          197.8927  401.2792  361.3629  1255.7428  ...   20.3899   
20          203.5040  390.5580  348.4122  1313.3912  ...   24.3124   
21          200.1983  388.3446  289.5213  1369.0847  ...   22.8586   
22          202.5682  388.7791  310.5086  1466.4543  ...   17.5252   

In [23]:
res = []
for col in avg_pred.columns:
    error = rmsse(truth[col].to_numpy(), avg_pred[col].to_numpy(),
                  df_sales_store_dept.loc[col].to_numpy())
    res.append(error)
    
df_res = pd.DataFrame({'RMSSE':res})
df_res.index = df_sales_store_dept.index
df_res

RMSSE
store_id dept_id            
CA_1     FOODS_1      1.2776
         FOODS_2      1.0254
         FOODS_3      1.3842
         HOBBIES_1    0.9252
         HOBBIES_2    1.2736
...                      ...
WI_3     FOODS_3      0.7947
         HOBBIES_1    0.9190
         HOBBIES_2    1.3275
         HOUSEHOLD_1  1.3616
         HOUSEHOLD_2  1.5959

[70 rows x 1 columns]

---

## Part 3. WRMSSE (10 pts.)

### Q5.  (10 pts.)

Calculate the `WRMSSE` for the all the methods described above. The weights can be found in `weights_validation.csv`.

For reference, the M5 benchmarks have the following `WRMSSE` scores at level 9:

- `Naive` = <b>1.764</b>


- `S.Naive` = <b>0.888</b>


- `ES_bu` = <b>0.728</b>

<i>Note: The M5 benchmarks use a bottom-up method for forecasting, so they will not necessarily be equal to your scores.</i>

In [24]:
df_weights = pd.read_csv('weights_validation.csv')
df_weights_9 = df_weights[df_weights.Level_id=='Level9']
df_weights_9

,Level_id,Agg_Level_1,Agg_Level_2,Weight
84,Level9,CA_1,FOODS_1,0.0052
85,Level9,CA_1,FOODS_2,0.0149
86,Level9,CA_1,FOODS_3,0.0419
87,Level9,CA_1,HOBBIES_1,0.0172
88,Level9,CA_1,HOBBIES_2,0.0006
...,...,...,...,...
149,Level9,WI_3,FOODS_3,0.0336
150,Level9,WI_3,HOBBIES_1,0.0072
151,Level9,WI_3,HOBBIES_2,0.0005
152,Level9,WI_3,HOUSEHOLD_1,0.0172


In [25]:
baseline_scores = []
for col in baselines.columns:
    baseline_scores.append(sum([i*j for i,j in zip(list(baselines[col]),
                                                  list(df_weights_9.Weight))]))
    
df_wrmsse = pd.DataFrame(baseline_scores, index=baselines.columns,
                         columns=['wrmsse'])

df_wrmsse.loc['onesteplgb'] = sum([i*j for i, j in zip(list(df_weights_9.Weight), 
                                                       list(onesteplgb_res.rmsse))])

df_wrmsse.loc['directlgb'] = sum([i*j for i, j in zip(list(df_weights_9.Weight),
                                                      list(direct_lgb_acc.RMSSE))])

df_wrmsse.loc['avg'] = sum([i*j for i, j in zip(list(df_weights_9.Weight),
                                                      df_res.RMSSE)])

df_wrmsse.T

,Naive,S. Naive,SES,Holt's Linear,Additive Holt-Winters,onesteplgb,directlgb,avg
wrmsse,1.6287,0.93,1.3425,1.2376,0.8442,2.8893,0.9249,1.7112


---

## Part 4. Middle-Out Method (30 pts.)

### Q6. Bottom-Up (15 pts.)

Using your forecasts from the best performing method in Q5, use the bottom-up method described in [FPP3](https://otexts.com/fpp3/single-level.html) to generate forecasts for levels 1 to 8.

Calculate the `WRMSSE` for levels 1 to 8 against the test set, then summarize the metrics in a dataframe and print it.

For reference, you can find the benchmark `WRMSSE` scores in the `The M5 Accuracy competition: Results, findings and conclusions` paper.

<i>Note: The M5 benchmarks use a bottom-up method for forecasting, so they will not necessarily be equal to your scores.</i>

The best performing model from Q5 is the Additive Holt-Winters model.

In [26]:
df_levels = pd.DataFrame()
df_levels[9] = list(df_wrmsse.T.min(1))

In [27]:
level_weights = {}

for i in range(1, 13):
    l_w = df_weights[df_weights.Level_id==f'Level{i}']
    level_weights[i] = list(l_w.Weight)
    display(l_w)

,Level_id,Agg_Level_1,Agg_Level_2,Weight
0,Level1,Total,X,1.0


,Level_id,Agg_Level_1,Agg_Level_2,Weight
1,Level2,CA,X,0.4424
2,Level2,TX,X,0.2693
3,Level2,WI,X,0.2883


,Level_id,Agg_Level_1,Agg_Level_2,Weight
4,Level3,CA_1,X,0.1109
5,Level3,CA_2,X,0.1102
6,Level3,CA_3,X,0.1556
7,Level3,CA_4,X,0.0656
8,Level3,TX_1,X,0.0776
9,Level3,TX_2,X,0.0952
10,Level3,TX_3,X,0.0965
11,Level3,WI_1,X,0.0871
12,Level3,WI_2,X,0.1166
13,Level3,WI_3,X,0.0846


,Level_id,Agg_Level_1,Agg_Level_2,Weight
14,Level4,FOODS,X,0.5686
15,Level4,HOBBIES,X,0.1281
16,Level4,HOUSEHOLD,X,0.3033


,Level_id,Agg_Level_1,Agg_Level_2,Weight
17,Level5,FOODS_1,X,0.0626
18,Level5,FOODS_2,X,0.1546
19,Level5,FOODS_3,X,0.3513
20,Level5,HOBBIES_1,X,0.1221
21,Level5,HOBBIES_2,X,0.0060
22,Level5,HOUSEHOLD_1,X,0.2296
23,Level5,HOUSEHOLD_2,X,0.0737


...

,Level_id,Agg_Level_1,Agg_Level_2,Weight
54,Level8,CA_1,FOODS,0.0620
55,Level8,CA_1,HOBBIES,0.0178
56,Level8,CA_1,HOUSEHOLD,0.0311
57,Level8,CA_2,FOODS,0.0603
58,Level8,CA_2,HOBBIES,0.0123
59,Level8,CA_2,HOUSEHOLD,0.0376
60,Level8,CA_3,FOODS,0.0815
61,Level8,CA_3,HOBBIES,0.0182
62,Level8,CA_3,HOUSEHOLD,0.0559
63,Level8,CA_4,FOODS,0.0369


,Level_id,Agg_Level_1,Agg_Level_2,Weight
84,Level9,CA_1,FOODS_1,0.0052
85,Level9,CA_1,FOODS_2,0.0149
86,Level9,CA_1,FOODS_3,0.0419
87,Level9,CA_1,HOBBIES_1,0.0172
88,Level9,CA_1,HOBBIES_2,0.0006
...,...,...,...,...
149,Level9,WI_3,FOODS_3,0.0336
150,Level9,WI_3,HOBBIES_1,0.0072
151,Level9,WI_3,HOBBIES_2,0.0005
152,Level9,WI_3,HOUSEHOLD_1,0.0172


,Level_id,Agg_Level_1,Agg_Level_2,Weight
154,Level10,FOODS_1_001,X,9.6200e-05
155,Level10,FOODS_1_002,X,2.7316e-04
156,Level10,FOODS_1_003,X,1.2409e-04
157,Level10,FOODS_1_004,X,0.0000e+00
158,Level10,FOODS_1_005,X,5.2040e-04
...,...,...,...,...
3198,Level10,HOUSEHOLD_2_512,X,1.8959e-04
3199,Level10,HOUSEHOLD_2_513,X,8.9000e-05
3200,Level10,HOUSEHOLD_2_514,X,2.3981e-04
3201,Level10,HOUSEHOLD_2_515,X,1.6800e-05


,Level_id,Agg_Level_1,Agg_Level_2,Weight
3203,Level11,CA,FOODS_1_001,5.8000e-05
3204,Level11,CA,FOODS_1_002,1.2140e-04
3205,Level11,CA,FOODS_1_003,7.8400e-05
3206,Level11,CA,FOODS_1_004,0.0000e+00
3207,Level11,CA,FOODS_1_005,2.9561e-04
...,...,...,...,...
12345,Level11,WI,HOUSEHOLD_2_512,4.4500e-05
12346,Level11,WI,HOUSEHOLD_2_513,1.2600e-05
12347,Level11,WI,HOUSEHOLD_2_514,6.2600e-05
12348,Level11,WI,HOUSEHOLD_2_515,6.3100e-06


,Level_id,Agg_Level_1,Agg_Level_2,Weight
12350,Level12,FOODS_1_001,CA_1,1.9700e-05
12351,Level12,FOODS_1_001,CA_2,1.8500e-05
12352,Level12,FOODS_1_001,CA_3,1.4300e-05
12353,Level12,FOODS_1_001,CA_4,5.3800e-06
12354,Level12,FOODS_1_001,TX_1,5.9800e-07
...,...,...,...,...
42835,Level12,HOUSEHOLD_2_516,TX_2,1.2700e-05
42836,Level12,HOUSEHOLD_2_516,TX_3,7.9200e-06
42837,Level12,HOUSEHOLD_2_516,WI_1,1.5800e-06
42838,Level12,HOUSEHOLD_2_516,WI_2,1.5800e-06


In [28]:
AHW = pd.DataFrame()

for i in list(df_sales_store_dept.index):
    AHW[i] = (pd.read_csv(f"baseline_predictions/pred_{i}.csv", index_col=0)
              .loc['Additive Holt-Winters'])

df_bu = AHW.T
df_bu.index = df_sales_store_dept.index
df_bu.reset_index(inplace=True)
df_bu

,store_id,dept_id,0,1,2,3,4,5,6,7,...,18,19,20,21,22,23,24,25,26,27
0,CA_1,FOODS_1,271.9093,267.0183,268.3435,280.7444,336.4040,392.1039,342.4206,272.2033,...,336.9919,392.6918,343.0085,272.7912,267.9001,269.2253,281.6263,337.2859,392.9857,343.3025
1,CA_1,FOODS_2,554.5889,486.3986,458.7023,441.8299,512.5000,647.5233,693.7097,554.6747,...,512.6716,647.6948,693.8813,554.8463,486.6560,458.9597,442.0873,512.7574,647.7806,693.9671
2,CA_1,FOODS_3,2129.1892,1909.9756,1890.0112,1878.8061,2160.3251,2743.1909,2892.2145,2130.8347,...,2163.6162,2746.4820,2895.5056,2134.1258,1914.9122,1894.9478,1883.7428,2165.2618,2748.1275,2897.1511
3,CA_1,HOBBIES_1,467.1562,445.1531,440.3657,440.6902,505.7043,598.1916,533.2492,467.4547,...,506.3012,598.7885,533.8461,468.0516,446.0484,441.2610,441.5855,506.5996,599.0869,534.1445
4,CA_1,HOBBIES_2,39.1266,38.8266,40.4819,39.1361,43.9363,50.3735,52.2296,39.2137,...,44.1105,50.5477,52.4039,39.3879,39.0879,40.7432,39.3974,44.1976,50.6348,52.4910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,WI_3,FOODS_3,1741.0084,1712.5839,1667.6337,1732.7465,1967.6493,2356.0623,2238.0090,1740.4405,...,1966.5135,2354.9265,2236.8731,1739.3046,1710.8801,1665.9299,1731.0427,1965.9455,2354.3585,2236.3052
66,WI_3,HOBBIES_1,226.0479,217.1481,226.9877,220.3259,285.5820,290.5686,247.0264,226.3232,...,286.1326,291.1192,247.5770,226.8738,217.9740,227.8136,221.1519,286.4079,291.3945,247.8523
67,WI_3,HOBBIES_2,29.3531,30.4989,31.0087,31.2690,32.3741,33.1313,32.2408,29.4199,...,32.5076,33.2648,32.3743,29.5534,30.6992,31.2090,31.4693,32.5744,33.3316,32.4411
68,WI_3,HOUSEHOLD_1,584.5345,547.5767,534.2372,568.1663,702.8680,821.8711,754.8438,585.3848,...,704.5686,823.5716,756.5443,587.0853,550.1275,536.7880,570.7171,705.4189,824.4219,757.3946


# Level 8

*All subsequent outputs are hidden but available upon clicking/expanding output*

In [29]:
ts_8 = df_sales_store_dept.reset_index()
ts_8['dept_id'] = ts_8['dept_id'].str[:-2]
ts_8 = ts_8.groupby(['store_id', 'dept_id']).sum()

t_8 = truth.T.reset_index()
t_8['dept_id'] = t_8['dept_id'].str[:-2]
t_8 = t_8.groupby(['store_id', 'dept_id']).sum()

p_8 = df_bu.copy()
p_8['dept_id'] = p_8['dept_id'].str[:-2]
p_8 = p_8.groupby(['store_id', 'dept_id']).sum()

display(ts_8, t_8, p_8)

rmsse_8 = []

for row in t_8.index:
    rmsse_8.append(rmsse(t_8.loc[row].to_numpy(), p_8.loc[row].to_numpy(),
                         ts_8.loc[row].to_numpy()))
    
df_levels[8] = [sum([i*j for i,j in zip(rmsse_8, level_weights[8])])]

2011-01-29  2011-01-30  2011-01-31  2011-02-01  \
store_id dept_id                                                     
CA_1     FOODS          3239.0      3137.0   2008.0000   2258.0000   
         HOBBIES         556.0       514.2    431.4000    402.6000   
         HOUSEHOLD       542.0       520.0    393.0000    401.0000   
CA_2     FOODS          2193.0      1921.0   1289.0000   1540.0000   
         HOBBIES         538.0       397.0    368.0000    358.5000   
         HOUSEHOLD       763.0       728.0    464.0000    434.0000   
CA_3     FOODS          3446.0      3535.0   2701.0000   3064.0000   
         HOBBIES         550.0       430.0    438.0000    424.0000   
         HOUSEHOLD       743.0       862.0    646.0000    744.0000   
CA_4     FOODS          1223.0      1269.0    946.0000   1002.0000   
         HOBBIES         338.0       414.0    363.0000    350.0000   
         HOUSEHOLD       244.0       272.0    189.0000    199.0000   
TX_1     FOODS          1781.0      1903.0   1377.0000   1643.0000   
         HOBBIES         251.0       264.0    107.0000    248.0000   
         HOUSEHOLD       524.0       520.0    338.0000    367.0000   
TX_2     FOODS          2788.0      2870.0   2082.0000   2180.0000   
         HOBBIES         385.0       359.0    229.0000    359.0000   
         HOUSEHOLD       679.0       708.0    420.0000    415.0000   
TX_3     FOODS          2284.0      2257.0   1665.0000   1647.0000   
         HOBBIES         243.0       247.0    190.0000    225.5000   
         HOUSEHOLD       503.0       502.0    370.0000    320.0000   
WI_1     FOODS          1581.0      1327.0    977.0000    935.0000   
         HOBBIES         615.0       443.0    327.3333    141.6667   
         HOUSEHOLD       508.0       424.0    262.0000    179.0000   
WI_2     FOODS          1615.0      1433.0   1586.0000   2013.0000   
         HOBBIES         190.0       134.0    113.0000    124.0000   
         HOUSEHOLD       451.0       362.0    319.0000    385.0000   
WI_3     FOODS          3028.0      3106.0   2543.0000   2596.0000   
         HOBBIES         278.0       356.0    248.0000    194.0000   
         HOUSEHOLD       732.0       736.0    526.0000    421.0000   

                    2011-02-02  2011-02-03  2011-02-04  2011-02-05  \
store_id dept_id                                                     
CA_1     FOODS       2032.0000   2407.0000   2693.0000      3962.0   
         HOBBIES      279.8000    467.0000    355.5000       769.0   
         HOUSEHOLD    330.0000    402.0000    411.0000       706.0   
CA_2     FOODS       1278.0000   1494.0000   1785.0000      2385.0   
         HOBBIES      296.0000    393.5000    316.0000       413.0   
         HOUSEHOLD    368.0000    403.0000    528.0000       931.0   
CA_3     FOODS       2761.0000   3340.0000   3455.0000      4068.0   
         HOBBIES      364.0000    398.5000    493.0000       525.0   
         HOUSEHOLD    692.0000    639.0000    755.0000       863.0   
CA_4     FOODS       1107.0000   1015.0000   1072.0000      1455.0   
         HOBBIES      352.0000    320.0000    318.0000       452.0   
         HOUSEHOLD    176.0000    163.0000    238.0000       254.0   
TX_1     FOODS       1329.0000   2091.0000   1247.0000      2008.0   
         HOBBIES      102.0000    297.6667     91.3333       264.0   
         HOUSEHOLD    263.0000    351.0000    355.0000       548.0   
TX_2     FOODS       1890.0000   2696.0000   1963.0000      2890.0   
         HOBBIES      257.0000    318.0000    162.0000       270.0   
         HOUSEHOLD    345.0000    425.0000    463.0000       612.0   
TX_3     FOODS       1383.0000   2280.0000   1461.0000      2157.0   
         HOBBIES      142.0000    230.3333    150.6667       251.0   
         HOUSEHOLD    201.0000    332.0000    347.0000       440.0   
WI_1     FOODS       1106.5000   1278.0000   1671.0000      2016.0   
         HOBBIES      306.5000    471.3333    737.6667       640.0   
         HOUSEHOLD    241.000

d_1914  d_1915  d_1916  d_1917  d_1918  d_1919  d_1920  \
store_id dept_id                                                             
CA_1     FOODS        2893    2518    2467    2446    3032    3950    4254   
         HOBBIES       589     426     451     432     584     636     684   
         HOUSEHOLD     990     759     797     740     957    1312    1459   
CA_2     FOODS        2654    2306    2224    2457    2861    4041    4017   
         HOBBIES       361     336     360     400     438     594     579   
         HOUSEHOLD     911     883     943     897    1083    1714    1597   
CA_3     FOODS        4106    3182    3234    3115    3503    4315    4878   
         HOBBIES       559     636     479     536     556     707     773   
         HOUSEHOLD    1694    1471    1352    1364    1646    2035    2177   
CA_4     FOODS        1841    1692    1616    1467    1718    1930    2291   
         HOBBIES       407     288     443     362     288     520     325   
         HOUSEHOLD     519     515     470     448     514     573     655   
TX_1     FOODS        2003    1790    1950    1623    2276    2543    2769   
         HOBBIES       253     237     263     292     286     420     378   
         HOUSEHOLD     820     826     771     749    1125    1089    1265   
TX_2     FOODS        2524    2319    2219    2173    2634    2877    3239   
         HOBBIES       404     418     325     366     420     522     451   
         HOUSEHOLD     955     765     712     902     969    1109    1182   
TX_3     FOODS        2427    2372    2184    2282    2846    2531    3267   
         HOBBIES       325     352     406     431     446     520     485   
         HOUSEHOLD     951     854     745     837    1160    1083    1157   
WI_1     FOODS        2209    2135    2217    2191    2774    3676    3761   
         HOBBIES       394     472     447     441     433     550     499   
         HOUSEHOLD     563     587     603     569     936    1092     968   
WI_2     FOODS        2933    2968    2692    3011    3564    3645    3753   
         HOBBIES       261     222     284     272     358     425     306   
         HOUSEHOLD     984     958     829    1059    1797    1415    1289   
WI_3     FOODS        2363    2237    2069    2150    2460    3215    3140   
         HOBBIES       229     258     272     210     276     320     378   
         HOUSEHOLD     671     705     621     510     956    1070    1056   

                    d_1921  d_1922  d_1923  ...  d_1932  d_1933  d_1934  \
store_id dept_id                            ...                           
CA_1     FOODS        3278    2997    2840  ...    3365    4046    4717   
         HOBBIES       529     628     565  ...     588     806     687   
         HOUSEHOLD     987     969     829  ...     998    1393    1303   
CA_2     FOODS        2995    2624    3015  ...    3255    3972    4285   
         HOBBIES       360     377     350  ...     370     687     588   
         HOUSEHOLD    1079     976     878  ...    1214    1848    1837   
CA_3     FOODS        4158    3883    4113  ...    3864    4311    4980   
         HOBBIES       643     661     574  ...     502     665     713   
         HOUSEHOLD    1975    1781    1517  ...    1680    2053    2276   
CA_4     FOODS        2019    1787    1707  ...    1667    2118    2215   
         HOBBIES       462     587     323  ...     371     480     413   
         HOUSEHOLD     632     560     555  ...     597     658     690   
TX_1     FOODS        1750    2008    1779  ...    2403    2528    3136   
         HOBBIES       214     278     259  ...     287     234     390   
         HOUSEHOLD     895     931     801  ...     920    1025    1474   
TX_2     FOODS        2492    2652    2381  ...    2964    2889    3887   
         HOBBIES       443     429     376  ...     381     374     556   
         HOUSEHOLD     896    1007     855  ...    1070    1161    1423   
TX_3     FOODS        2

0          1          2          3          4  \
store_id dept_id                                                            
CA_1     FOODS      2955.6874  2663.3924  2617.0570  2601.3805  3009.2291   
         HOBBIES     506.2828   483.9797   480.8476   479.8263   549.6406   
         HOUSEHOLD   964.8373   820.8803   791.7922   799.9078   933.6542   
CA_2     FOODS      2589.8796  2500.8731  2462.5473  2471.2071  2915.6517   
         HOBBIES     375.1397   363.6798   366.7274   369.3618   424.9453   
         HOUSEHOLD   955.9834   868.2458   877.5887   873.0851  1072.2209   
CA_3     FOODS      3833.6402  3513.9347  3382.1971  3359.7706  3562.1404   
         HOBBIES     550.8618   538.2662   540.5118   533.9557   573.8770   
         HOUSEHOLD  1791.5102  1618.7566  1546.3182  1542.9067  1651.8203   
CA_4     FOODS      1831.5078  1665.0927  1630.9272  1651.8652  1729.7034   
         HOBBIES     369.0445   371.7976   354.4030   358.8456   362.3282   
         HOUSEHOLD   563.2823   522.9140   515.3003   513.6505   524.4852   
TX_1     FOODS      1972.5404  1825.9584  1819.3318  1826.2057  1948.7119   
         HOBBIES     265.6561   252.9745   253.5305   258.6317   287.1208   
         HOUSEHOLD   871.0637   803.1119   791.3602   799.5544   875.1489   
TX_2     FOODS      2419.1661  2165.1613  2160.1163  2226.1396  2420.0711   
         HOBBIES     374.7260   339.0247   350.9436   353.5970   379.9638   
         HOUSEHOLD   902.0970   810.9132   823.5214   853.8364   941.1867   
TX_3     FOODS      2285.3761  2083.6172  2037.4334  2061.2002  2206.6591   
         HOBBIES     390.2367   371.3845   372.5775   372.5353   412.7808   
         HOUSEHOLD   897.4615   820.5453   798.3788   827.6824   900.0080   
WI_1     FOODS      2212.1769  2115.4667  2182.8806  2235.1109  2730.5688   
         HOBBIES     329.4407   337.2679   353.6799   354.1591   458.0034   
         HOUSEHOLD   620.5356   560.0326   605.0708   632.2399   822.1825   
WI_2     FOODS      3266.6280  3207.4974  3225.0155  3247.1796  3422.2147   
         HOBBIES     266.4352   258.4884   270.9137   273.9175   306.7200   
         HOUSEHOLD  1042.2649  1032.0818  1079.6596  1147.8631  1302.1347   
WI_3     FOODS      2438.3661  2429.5206  2351.7396  2427.3105  2719.9078   
         HOBBIES     255.4010   247.6470   257.9964   251.5949   317.9560   
         HOUSEHOLD   738.3618   696.5243   682.3169   721.7583   881.8567   

                            5          6          7          8          9  \
store_id dept_id                                                            
CA_1     FOODS      3782.8181  3928.3448  2957.7127  2665.4177  2619.0823   
         HOBBIES     648.5651   585.4788   506.6684   484.3652   481.2332   
         HOUSEHOLD  1289.0636  1312.3824   966.6836   822.7267   793.6385   
CA_2     FOODS      3793.8219  3813.1109  2595.6125  2506.6060  2468.2802   
         HOBBIES     529.0281   468.6417   375.7951   364.3352   367.3828   
         HOUSEHOLD  1527.3640  1528.6383   958.1979   870.4603   879.8032   
CA_3     FOODS      4378.5887  4673.4277  3836.2429  3516.5374  3384.7998   
         HOBBIES     670.0939   697.4333   551.3399   538.7443   540.9900   
         HOUSEHOLD  2126.2445  2232.2405  1795.2993  1622.5457  1550.1072   
CA_4     FOODS      1956.8455  2028.6026  1834.0433  1667.6282  1633.4627   
         HOBBIES     409.3632   390.2794   369.5529   372.3060   354.9114   
         HOUSEHOLD   616.8195   637.7088   564.6697   524.3014   516.6877   
TX_1     FOODS      2356.4153  2536.4542  1974.1811  1827.5991  1820.9725   
         HOBBIES     324.7860   337.1696   266.0472   253.3655   253.9215   
         HOUSEHOLD  1095.1861  1140.4148   872.9152   804.9634   793.2116   
TX_2     FOODS      2863.1319  3059.2443  2419.4492  2165.4444  2160.3994   
         HOBBIES     420.0623   430.7099   375.1122   339.4109   351.3298   
         HOUSEHOLD  1166.4888  1152.2150   903.5789   812.3951   825.0033   
TX_3     FOODS      255

# Level 7

In [30]:
ts_7 = df_sales_store_dept.reset_index()
ts_7['store_id'] = ts_7['store_id'].str[:-2]
ts_7 = ts_7.groupby(['store_id', 'dept_id']).sum()

t_7 = truth.T.reset_index()
t_7['store_id'] = t_7['store_id'].str[:-2]
t_7 = t_7.groupby(['store_id', 'dept_id']).sum()

p_7 = df_bu.copy()
p_7['store_id'] = p_7['store_id'].str[:-2]
p_7 = p_7.groupby(['store_id', 'dept_id']).sum()

display(ts_7, t_7, p_7)

rmsse_7 = []

for row in t_7.index:
    rmsse_7.append(rmsse(t_7.loc[row].to_numpy(), p_7.loc[row].to_numpy(),
                         ts_7.loc[row].to_numpy()))
    
df_levels[7] = [sum([i*j for i,j in zip(rmsse_7, level_weights[7])])]

2011-01-29  2011-01-30  2011-01-31  2011-02-01  \
store_id dept_id                                                       
CA       FOODS_1          1157.0      1142.0    787.0000    743.0000   
         FOODS_2          1864.0      1987.0   1315.0000   1452.0000   
         FOODS_3          7080.0      6733.0   4842.0000   5669.0000   
         HOBBIES_1        1739.0      1502.0   1422.0000   1352.0000   
         HOBBIES_2         243.0       253.2    178.4000    183.1000   
         HOUSEHOLD_1      1527.0      1515.0   1116.0000   1148.0000   
         HOUSEHOLD_2       765.0       867.0    576.0000    630.0000   
TX       FOODS_1           500.0       446.0    381.0000    317.0000   
         FOODS_2          1384.0      1426.0   1008.0000   1316.0000   
         FOODS_3          4969.0      5158.0   3735.0000   3837.0000   
         HOBBIES_1         837.0       775.0    435.0000    756.0000   
         HOBBIES_2          42.0        95.0     91.0000     76.5000   
         HOUSEHOLD_1      1281.0      1261.0    845.0000    820.0000   
         HOUSEHOLD_2       425.0       469.0    283.0000    282.0000   
WI       FOODS_1           686.0       628.0    489.0000    448.0000   
         FOODS_2           846.0       796.0    851.0000    838.0000   
         FOODS_3          4692.0      4442.0   3766.0000   4258.0000   
         HOBBIES_1        1034.0       895.0    640.0000    423.0000   
         HOBBIES_2          49.0        38.0     48.3333     36.6667   
         HOUSEHOLD_1      1297.0      1082.0    866.0000    764.0000   
         HOUSEHOLD_2       394.0       440.0    241.0000    221.0000   

                      2011-02-02  2011-02-03  2011-02-04  2011-02-05  \
store_id dept_id                                                       
CA       FOODS_1        729.0000       796.0    913.0000      1036.0   
         FOODS_2       1239.0000      1427.0   1554.0000      1962.0   
         FOODS_3       5210.0000      6033.0   6538.0000      8872.0   
         HOBBIES_1     1139.0000      1427.0   1281.0000      1920.0   
         HOBBIES_2      152.8000       152.0    201.5000       239.0   
         HOUSEHOLD_1    935.0000      1014.0   1224.0000      1834.0   
         HOUSEHOLD_2    631.0000       593.0    708.0000       920.0   
TX       FOODS_1        297.0000       445.0    337.0000       466.0   
         FOODS_2       1143.0000      1826.0    917.0000      1319.0   
         FOODS_3       3162.0000      4796.0   3417.0000      5270.0   
         HOBBIES_1      448.0000       798.0    355.0000       732.0   
         HOBBIES_2       53.0000        48.0     49.0000        53.0   
         HOUSEHOLD_1    598.0000       801.0    843.0000      1142.0   
         HOUSEHOLD_2    211.0000       307.0    322.0000       458.0   
WI       FOODS_1        404.0000       656.0    653.0000       733.0   
         FOODS_2        543.0000      1122.0    878.0000      1103.0   
         FOODS_3       2980.5000      4992.0   5283.0000      6990.0   
         HOBBIES_1      422.5000       908.0   1219.0000      1179.0   
         HOBBIES_2       35.8333        35.0     34.1667        36.0   
         HOUSEHOLD_1    434.0000       849.0   1094.0000      1358.0   
         HOUSEHOLD_2    161.0000       334.0    385.0000       483.0   

                      2011-02-06  2011-02-07  ...  2016-04-15  2016-04-16  \
store_id dept_id                              ...                           
CA       FOODS_1       1049.0000    804.0000  ...      1532.0      1562.0   
         FOODS_2       1928.0000   1787.0000  ...      1656.0      2091.0   
         FOODS_3       8000.0000   6046.0000  ...      7544.0      9441.0   
         HOBBIES_1     1419.0000   1471.0000  ...      1496.0      2041.0   
         HOBBIES_2      197.3333    177.6667  ...       162.0       135.0   
         HOUSEHOLD_1   1496.0000   1089.0000  ...      2984.0      4064.0   
         HOUSEHOLD_2    741.0000    588.0000  ...       881.0      1230.0   
TX       FOODS_

d_1914  d_1915  d_1916  d_1917  d_1918  d_1919  d_1920  \
store_id dept_id                                                               
CA       FOODS_1        1417    1275    1352    1440    1506    1960    1914   
         FOODS_2        1993    1902    1605    1505    1827    2361    2941   
         FOODS_3        8084    6521    6584    6540    7781    9915   10585   
         HOBBIES_1      1746    1505    1557    1509    1686    2150    2090   
         HOBBIES_2       170     181     176     221     180     307     271   
         HOUSEHOLD_1    3177    2736    2728    2602    3155    4317    4483   
         HOUSEHOLD_2     937     892     834     847    1045    1317    1405   
TX       FOODS_1         653     683     703     752     964     946     964   
         FOODS_2        1227    1042     908     967    1254    1106    1597   
         FOODS_3        5074    4756    4742    4359    5538    5899    6714   
         HOBBIES_1       846     864     794     922     995    1320    1161   
         HOBBIES_2       136     143     200     167     157     142     153   
         HOUSEHOLD_1    2246    1958    1764    1985    2636    2597    2861   
         HOUSEHOLD_2     480     487     464     503     618     684     743   
WI       FOODS_1         771     902     744     992    1330    1509    1407   
         FOODS_2        1737    1718    1598    1617    1892    2314    2591   
         FOODS_3        4997    4720    4636    4743    5576    6713    6656   
         HOBBIES_1       797     848     885     822     968    1157    1071   
         HOBBIES_2        87     104     118     101      99     138     112   
         HOUSEHOLD_1    1844    1820    1672    1785    3163    2961    2755   
         HOUSEHOLD_2     374     430     381     353     526     616     558   

                      d_1921  d_1922  d_1923  ...  d_1932  d_1933  d_1934  \
store_id dept_id                              ...                           
CA       FOODS_1        1535    1614    1598  ...    1931    1854    1820   
         FOODS_2        2433    2052    2067  ...    2108    2396    3130   
         FOODS_3        8482    7625    8010  ...    8112   10197   11247   
         HOBBIES_1      1811    2078    1610  ...    1643    2386    2195   
         HOBBIES_2       183     175     202  ...     188     252     206   
         HOUSEHOLD_1    3621    3305    2911  ...    3378    4398    4598   
         HOUSEHOLD_2    1052     981     868  ...    1111    1554    1508   
TX       FOODS_1         849    1046     773  ...    1022     924    1025   
         FOODS_2        1293    1448     963  ...    1320    1339    1914   
         FOODS_3        4679    5184    4659  ...    6027    5832    7724   
         HOBBIES_1       907    1001     775  ...     872     827    1253   
         HOBBIES_2       193     162     197  ...     156     106     208   
         HOUSEHOLD_1    2247    2529    2012  ...    2347    2717    3353   
         HOUSEHOLD_2     551     579     530  ...     576     621     772   
WI       FOODS_1        1122    1360    1231  ...    1177    1479    1380   
         FOODS_2        2658    2796    2380  ...    2414    3828    4025   
         FOODS_3        6091    6469    5518  ...    5996    8727    9182   
         HOBBIES_1       877     828     805  ...     907    1201    1016   
         HOBBIES_2        88     129     119  ...     105     121     138   
         HOUSEHOLD_1    2111    2478    1929  ...    2211    2927    2621   
         HOUSEHOLD_2     398     475     444  ...     472     622     606   

                      d_1935  d_1936  d_1937  d_1938  d_1939  d_1940  d_1941  
store_id dept_id                                                              
CA       FOODS_1        1408    1316    1374    1364    1631    2095    2041  
         FOODS_2        2203    1955    1736    1795    1805    2421    2888  
         FOODS_3        8248    7488    7355    7491    7918    9973   11041  
         HOBBI

0          1          2          3          4  \
store_id dept_id                                                              
CA       FOODS_1      1210.1178  1204.8121  1211.2006  1272.1528  1448.3923   
         FOODS_2      2146.3783  1918.3955  1846.9577  1757.0236  1914.4956   
         FOODS_3      7854.2189  7220.0852  7034.5702  7055.0470  7853.8368   
         HOBBIES_1    1657.1367  1612.2800  1593.9019  1595.0106  1753.1771   
         HOBBIES_2     144.1920   145.4432   148.5878   146.9788   157.6140   
         HOUSEHOLD_1  3307.8778  2931.5683  2827.8651  2821.0508  3161.7925   
         HOUSEHOLD_2   967.7354   899.2285   903.1342   908.4992  1020.3880   
TX       FOODS_1       571.9197   588.1070   619.4685   653.5809   707.6242   
         FOODS_2      1185.3391  1046.2444  1011.9019   992.5232  1021.6248   
         FOODS_3      4919.8237  4440.3855  4385.5112  4467.4414  4846.1930   
         HOBBIES_1     900.1884   833.1765   840.3496   850.1440   949.6690   
         HOBBIES_2     130.4304   130.2073   136.7020   134.6200   130.1963   
         HOUSEHOLD_1  2140.4579  1930.3328  1906.8217  1960.1856  2151.5768   
         HOUSEHOLD_2   530.1643   504.2377   506.4386   520.8877   564.7667   
WI       FOODS_1       827.3554   893.3974   886.3812   906.4724  1031.1561   
         FOODS_2      1887.3206  1798.6218  1762.8743  1750.8495  1931.5096   
         FOODS_3      5202.4950  5060.4655  5110.3802  5252.2791  5910.0256   
         HOBBIES_1     766.3755   759.3355   794.7631   790.8589   984.5999   
         HOBBIES_2      84.9014    84.0677    87.8270    88.8126    98.0796   
         HOUSEHOLD_1  1959.9288  1863.3442  1935.5418  2050.5375  2475.7758   
         HOUSEHOLD_2   441.2335   425.2944   431.5055   451.3238   530.3982   

                              5           6          7          8          9  \
store_id dept_id                                                               
CA       FOODS_1      1694.9447   1527.3229  1211.7659  1206.4603  1212.8487   
         FOODS_2      2434.2431   2689.5501  2148.5768  1920.5941  1849.1563   
         FOODS_3      9782.8863  10226.6129  7863.2687  7229.1350  7043.6200   
         HOBBIES_1    2081.0247   1963.4764  1658.8493  1613.9927  1595.6146   
         HOBBIES_2     176.0256    178.3568   144.5069   145.7581   148.9027   
         HOUSEHOLD_1  4277.1832   4431.6591  3315.4683  2939.1587  2835.4555   
         HOUSEHOLD_2  1282.3085   1279.3109   969.3822   900.8754   904.7810   
TX       FOODS_1       740.1778    680.4505   572.7484   588.9358   620.2972   
         FOODS_2      1213.3442   1381.2568  1184.8799  1045.7851  1011.4426   
         FOODS_3      5821.8004   6224.1598  4922.7761  4443.3378  4388.4635   
         HOBBIES_1    1065.1261   1087.1953   901.6167   834.6048   841.7779   
         HOBBIES_2     140.8804    146.3237   130.6806   130.4575   136.9522   
         HOUSEHOLD_1  2684.5570   2684.0362  2144.6975  1934.5724  1911.0614   
         HOUSEHOLD_2   670.8242    663.5758   531.1548   505.2281   507.4291   
WI       FOODS_1      1154.4509    974.4538   828.7801   894.8222   887.8060   
         FOODS_2      2294.5095   2304.2028  1891.4749  1802.7760  1767.0286   
         FOODS_3      7056.7234   6721.7163  5208.4343  5066.4048  5116.3194   
         HOBBIES_1    1141.9770    916.4737   767.6304   760.5904   796.0179   
         HOBBIES_2     104.9059     95.9898    85.0831    84.2494    88.0086   
         HOUSEHOLD_1  2865.7754   2488.9926  1964.6510  1868.0665  1940.2640   
         HOUSEHOLD_2   604.0333    555.0137   442.0869   426.1478   432.3588   

                      ...         18         19          20         21  \
store_id dept_id      ...                                                
CA       FOODS_1      ...  1451.6885  1698.2409   1530.6192  1215.0622   
         FOODS_2      ...  1918.8926  2438.6401   2693.9472  2152.9739   
         FOODS_3      ...  7871.9364  9800.9860  10244.7125  7881.3683   
 

# Level 6

In [31]:
ts_6 = df_sales_store_dept.reset_index()
ts_6['dept_id'] = ts_6['dept_id'].str[:-2]
ts_6['store_id'] = ts_6['store_id'].str[:-2]
ts_6 = ts_6.groupby(['store_id', 'dept_id']).sum()

t_6 = truth.T.reset_index()
t_6['dept_id'] = t_6['dept_id'].str[:-2]
t_6['store_id'] = t_6['store_id'].str[:-2]
t_6 = t_6.groupby(['store_id', 'dept_id']).sum()

p_6 = df_bu.copy()
p_6['dept_id'] = p_6['dept_id'].str[:-2]
p_6['store_id'] = p_6['store_id'].str[:-2]
p_6 = p_6.groupby(['store_id', 'dept_id']).sum()

display(ts_6, t_6, p_6)

rmsse_6 = []

for row in t_6.index:
    rmsse_6.append(rmsse(t_6.loc[row].to_numpy(), p_6.loc[row].to_numpy(),
                         ts_6.loc[row].to_numpy()))
    
df_levels[6] = [sum([i*j for i,j in zip(rmsse_6, level_weights[6])])]

2011-01-29  2011-01-30  2011-01-31  2011-02-01  \
store_id dept_id                                                     
CA       FOODS         10101.0      9862.0   6944.0000   7864.0000   
         HOBBIES        1982.0      1755.2   1600.4000   1535.1000   
         HOUSEHOLD      2292.0      2382.0   1692.0000   1778.0000   
TX       FOODS          6853.0      7030.0   5124.0000   5470.0000   
         HOBBIES         879.0       870.0    526.0000    832.5000   
         HOUSEHOLD      1706.0      1730.0   1128.0000   1102.0000   
WI       FOODS          6224.0      5866.0   5106.0000   5544.0000   
         HOBBIES        1083.0       933.0    688.3333    459.6667   
         HOUSEHOLD      1691.0      1522.0   1107.0000    985.0000   

                    2011-02-02  2011-02-03  2011-02-04  2011-02-05  \
store_id dept_id                                                     
CA       FOODS       7178.0000      8256.0   9005.0000     11870.0   
         HOBBIES     1291.8000      1579.0   1482.5000      2159.0   
         HOUSEHOLD   1566.0000      1607.0   1932.0000      2754.0   
TX       FOODS       4602.0000      7067.0   4671.0000      7055.0   
         HOBBIES      501.0000       846.0    404.0000       785.0   
         HOUSEHOLD    809.0000      1108.0   1165.0000      1600.0   
WI       FOODS       3927.5000      6770.0   6814.0000      8826.0   
         HOBBIES      458.3333       943.0   1253.1667      1215.0   
         HOUSEHOLD    595.0000      1183.0   1479.0000      1841.0   

                    2011-02-06  2011-02-07  ...  2016-04-15  2016-04-16  \
store_id dept_id                            ...                           
CA       FOODS      10977.0000   8637.0000  ...     10732.0     13094.0   
         HOBBIES     1616.3333   1648.6667  ...      1658.0      2176.0   
         HOUSEHOLD   2237.0000   1677.0000  ...      3865.0      5294.0   
TX       FOODS       6920.0000   5505.0000  ...      6994.0      7944.0   
         HOBBIES      794.0000    539.5000  ...      1107.0      1124.0   
         HOUSEHOLD   1662.0000   1290.0000  ...      2699.0      3392.0   
WI       FOODS       6965.0000   4759.0000  ...     10956.0     10969.0   
         HOBBIES      641.5000    594.5000  ...      1021.0      1334.0   
         HOUSEHOLD   1076.0000   1089.0000  ...      2757.0      3035.0   

                    2016-04-17  2016-04-18  2016-04-19  2016-04-20  \
store_id dept_id                                                     
CA       FOODS         14855.0     11217.0     10088.0      9863.0   
         HOBBIES        2330.0      1706.0      1849.0      1646.0   
         HOUSEHOLD      5847.0      4129.0      3847.0      3639.0   
TX       FOODS          8717.0      6834.0      7066.0      6242.0   
         HOBBIES        1461.0       830.0      1025.0       980.0   
         HOUSEHOLD      3531.0      2204.0      2831.0      2378.0   
WI       FOODS         10925.0      8100.0      7794.0      7527.0   
         HOBBIES        1029.0       787.0       913.0       846.0   
         HOUSEHOLD      2945.0      2252.0      2157.0      2222.0   

                    2016-04-21  2016-04-22  2016-04-23  2016-04-24  
store_id dept_id                                                    
CA       FOODS          9479.0     10991.0     13845.0     14806.0  
         HOBBIES        1590.0      2015.0      2367.0      2357.0  
         HOUSEHOLD      3419.0      4089.0      5622.0      6024.0  
TX       FOODS          6167.0      6839.0      7849.0      7765.0  
         HOBBIES         882.0      1056.0      1222.0      1266.0  
         HOUSEHOLD      2553.0      2720.0      3195.0      3251.0  
WI       FOODS          7671.0      8874.0     10233.0     10083.0  
         HOBBIES         881.0      1014.0      1198.0      1060.0  
         HOUSEHOLD      2391.0      2919.0      3431.0      3183.0  

[9 rows x 1913 columns]

d_1914  d_1915  d_1916  d_1917  d_1918  d_1919  d_1920  \
store_id dept_id                                                             
CA       FOODS       11494    9698    9541    9485   11114   14236   15440   
         HOBBIES      1916    1686    1733    1730    1866    2457    2361   
         HOUSEHOLD    4114    3628    3562    3449    4200    5634    5888   
TX       FOODS        6954    6481    6353    6078    7756    7951    9275   
         HOBBIES       982    1007     994    1089    1152    1462    1314   
         HOUSEHOLD    2726    2445    2228    2488    3254    3281    3604   
WI       FOODS        7505    7340    6978    7352    8798   10536   10654   
         HOBBIES       884     952    1003     923    1067    1295    1183   
         HOUSEHOLD    2218    2250    2053    2138    3689    3577    3313   

                    d_1921  d_1922  d_1923  ...  d_1932  d_1933  d_1934  \
store_id dept_id                            ...                           
CA       FOODS       12450   11291   11675  ...   12151   14447   16197   
         HOBBIES      1994    2253    1812  ...    1831    2638    2401   
         HOUSEHOLD    4673    4286    3779  ...    4489    5952    6106   
TX       FOODS        6821    7678    6395  ...    8369    8095   10663   
         HOBBIES      1100    1163     972  ...    1028     933    1461   
         HOUSEHOLD    2798    3108    2542  ...    2923    3338    4125   
WI       FOODS        9871   10625    9129  ...    9587   14034   14587   
         HOBBIES       965     957     924  ...    1012    1322    1154   
         HOUSEHOLD    2509    2953    2373  ...    2683    3549    3227   

                    d_1935  d_1936  d_1937  d_1938  d_1939  d_1940  d_1941  
store_id dept_id                                                            
CA       FOODS       11859   10759   10465   10650   11354   14489   15970  
         HOBBIES      1554    1649    1606    1735    1975    2701    2514  
         HOUSEHOLD    4308    3742    3607    3912    4101    5913    6160  
TX       FOODS        7905    7164    6596    6083    7965    8834    9500  
         HOBBIES      1205    1199    1056     830    1132    1440    1543  
         HOUSEHOLD    3118    3007    2723    2249    3206    3407    3772  
WI       FOODS        9394    8225    7729    8004    8817   10276   10497  
         HOBBIES       769     781    1078     910    1036    1192    1223  
         HOUSEHOLD    2250    2251    2236    2590    2966    3266    3159  

[9 rows x 28 columns]

0           1           2           3  \
store_id dept_id                                                     
CA       FOODS      11210.7150  10343.2929  10092.7285  10084.2234   
         HOBBIES     1801.3287   1757.7232   1742.4898   1741.9894   
         HOUSEHOLD   4275.6133   3830.7968   3730.9993   3729.5500   
TX       FOODS       6677.0826   6074.7369   6016.8815   6113.5455   
         HOBBIES     1030.6188    963.3838    977.0516    984.7640   
         HOUSEHOLD   2670.6222   2434.5704   2413.2604   2481.0732   
WI       FOODS       7917.1710   7752.4847   7759.6357   7909.6011   
         HOBBIES      851.2769    843.4033    882.5900    879.6715   
         HOUSEHOLD   2401.1623   2288.6387   2367.0472   2501.8613   

                             4           5           6           7  \
store_id dept_id                                                     
CA       FOODS      11216.7247  13912.0741  14443.4860  11223.6114   
         HOBBIES     1910.7911   2257.0503   2141.8332   1803.3563   
         HOUSEHOLD   4182.1805   5559.4916   5710.9700   4284.8505   
TX       FOODS       6575.4420   7775.3224   8285.8672   6680.4044   
         HOBBIES     1079.8653   1206.0065   1233.5191   1032.2974   
         HOUSEHOLD   2716.3435   3355.3812   3347.6120   2675.8523   
WI       FOODS       8872.6913  10505.6839  10000.3729   7928.6893   
         HOBBIES     1082.6795   1246.8829   1012.4634    852.7135   
         HOUSEHOLD   3006.1740   3469.8087   3044.0063   2406.7379   

                             8           9  ...          18          19  \
store_id dept_id                            ...                           
CA       FOODS      10356.1893  10105.6250  ...  11242.5176  13937.8670   
         HOBBIES     1759.7508   1744.5173  ...   1914.8462   2261.1054   
         HOUSEHOLD   3840.0341   3740.2366  ...   4200.6550   5577.9661   
TX       FOODS       6078.0587   6020.2033  ...   6582.0857   7781.9660   
         HOBBIES      965.0623    978.7301  ...   1083.2224   1209.3636   
         HOUSEHOLD   2439.8006   2418.4905  ...   2726.8038   3365.8415   
WI       FOODS       7764.0030   7771.1539  ...   8895.7277  10528.7203   
         HOBBIES      844.8398    884.0266  ...   1085.5526   1249.7560   
         HOUSEHOLD   2294.2143   2372.6228  ...   3017.3251   3480.9599   

                            20          21          22          23  \
store_id dept_id                                                     
CA       FOODS      14469.2789  11249.4043  10381.9822  10131.4179   
         HOBBIES     2145.8883   1807.4114   1763.8059   1748.5724   
         HOUSEHOLD   5729.4445   4303.3250   3858.5086   3758.7111   
TX       FOODS       8292.5108   6687.0480   6084.7023   6026.8470   
         HOBBIES     1236.8761   1035.6544    968.4193    982.0871   
         HOUSEHOLD   3358.0723   2686.3126   2450.2608   2428.9508   
WI       FOODS      10023.4093   7951.7257   7787.0394   7794.1904   
         HOBBIES     1015.3365    855.5866    847.7130    886.8997   
         HOUSEHOLD   3055.1575   2417.8891   2305.3654   2383.7740   

                            24          25          26          27  
store_id dept_id                                                    
CA       FOODS      10122.9128  11255.4140  13950.7635  14482.1753  
         HOBBIES     1748.0720   1916.8737   2263.1329   2147.9158  
         HOUSEHOLD   3757.2618   4209.8923   5587.2034   5738.6818  
TX       FOODS       6123.5109   6585.4075   7785.2878   8295.8326  
         HOBBIES      989.7996   1084.9009   1211.0421   1238.5546  
         HOUSEHOLD   2496.7636   2732.0339   3371.0716   3363.3024  
WI       FOODS       7944.1557   8907.2460  10540.2386  10034.9276  
         HOBBIES      883.9812   1086.9892   1251.1926   1016.7731  
         HOUSEHOLD   2518.5881   3022.9007   3486.5354   3060.7331  

[9 rows x 28 columns]

# Level 5

In [32]:
ts_5 = df_sales_store_dept.reset_index()
ts_5 = ts_5.groupby(['dept_id']).sum()

t_5 = truth.T.reset_index()
t_5 = t_5.groupby(['dept_id']).sum()

p_5 = df_bu.copy()
p_5 = p_5.groupby(['dept_id']).sum()

display(ts_5, t_5, p_5)

rmsse_5 = []

for row in t_5.index:
    rmsse_5.append(rmsse(t_5.loc[row].to_numpy(), p_5.loc[row].to_numpy(),
                         ts_5.loc[row].to_numpy()))
    
df_levels[5] = [sum([i*j for i,j in zip(rmsse_5, level_weights[5])])]

,2011-01-29,2011-01-30,2011-01-31,2011-02-01,2011-02-02,2011-02-03,2011-02-04,2011-02-05,2011-02-06,2011-02-07,...,2016-04-15,2016-04-16,2016-04-17,2016-04-18,2016-04-19,2016-04-20,2016-04-21,2016-04-22,2016-04-23,2016-04-24
dept_id,,,,,,,,,,,,,,,,,,,,,
FOODS_1,2343.0,2216.0,1657.0000,1508.0000,1430.0000,1897.0,1903.0000,2235.0,1925.0000,1586.0000,...,3192.0,3523.0,3062.0,2461.0,2544.0,2657.0,2639.0,3169.0,3712.0,3185.0
FOODS_2,4094.0,4209.0,3174.0000,3606.0000,2925.0000,4375.0,3349.0000,4384.0,4273.0000,3684.0000,...,5661.0,5792.0,6588.0,5247.0,4942.0,4734.0,4446.0,4974.0,5524.0,6473.0
FOODS_3,16741.0,16333.0,12343.0000,13764.0000,11352.5000,15821.0,15238.0000,21132.0,18664.0000,13631.0000,...,19829.0,22692.0,24847.0,18443.0,17462.0,16241.0,16232.0,18561.0,22691.0,22996.0
HOBBIES_1,3610.0,3172.0,2497.0000,2531.0000,2009.5000,3133.0,2855.0000,3831.0,2741.0000,2514.0000,...,3367.0,4220.0,4325.0,3038.0,3420.0,3141.0,3014.0,3678.0,4397.0,4226.0
HOBBIES_2,334.0,386.2,317.7333,296.2667,241.6333,235.0,284.6667,328.0,310.8333,268.6667,...,419.0,414.0,495.0,285.0,367.0,331.0,339.0,407.0,390.0,457.0
HOUSEHOLD_1,4105.0,3858.0,2827.0000,2732.0000,1967.0000,2664.0,3161.0000,4334.0,3494.0000,2857.0000,...,7375.0,9253.0,9720.0,6776.0,6986.0,6393.0,6580.0,7762.0,9545.0,9795.0
HOUSEHOLD_2,1584.0,1776.0,1100.0000,1133.0000,1003.0000,1234.0,1415.0000,1861.0,1481.0000,1199.0000,...,1946.0,2468.0,2603.0,1809.0,1849.0,1846.0,1783.0,1966.0,2703.0,2663.0


,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
dept_id,,,,,,,,,,,,,,,,,,,,,
FOODS_1,2841,2860,2799,3184,3800,4415,4285,3506,4020,3602,...,4130,4257,4225,3264,3119,3073,3251,3735,4195,4031
FOODS_2,4957,4662,4111,4089,4973,5781,7129,6384,6296,5410,...,5842,7563,9069,6226,5311,4648,4551,4815,5940,6535
FOODS_3,18155,15997,15962,15642,18895,22527,23955,19252,19278,18187,...,20135,24756,28153,19668,17718,17069,16935,19586,23464,25401
HOBBIES_1,3389,3217,3236,3253,3649,4627,4322,3595,3907,3190,...,3422,4414,4464,3109,3142,3291,3027,3761,4832,4720
HOBBIES_2,393,428,494,489,436,587,536,464,466,518,...,449,479,552,419,487,449,448,382,501,560
HOUSEHOLD_1,7267,6514,6164,6372,8954,9875,10099,7979,8312,6852,...,7936,10042,10572,7609,6999,6641,6759,7974,9668,10165
HOUSEHOLD_2,1791,1809,1679,1703,2189,2617,2706,2001,2035,1842,...,2159,2797,2886,2067,2001,1925,1992,2299,2918,2926


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
dept_id,,,,,,,,,,,,,,,,,,,,,
FOODS_1,2609.3929,2686.3166,2717.0503,2832.2061,3187.1725,3589.5734,3182.2273,2613.2945,2690.2182,2720.9519,...,3194.9757,3597.3766,3190.0305,2621.0977,2698.0214,2728.7551,2843.9109,3198.8773,3601.2782,3193.9321
FOODS_2,5219.0380,4763.2617,4621.7339,4500.3963,4867.6300,5942.0968,6375.0097,5224.9315,4769.1552,4627.6275,...,4879.4171,5953.8838,6386.7967,5236.7186,4780.9422,4639.4145,4518.0769,4885.3106,5959.7774,6392.6902
FOODS_3,17976.5376,16720.9362,16530.4615,16774.7676,18610.0554,22661.4102,23172.4891,17994.4790,16738.8776,16548.4029,...,18645.9381,22697.2929,23208.3718,18030.3618,16774.7604,16584.2857,16828.5917,18663.8795,22715.2343,23226.3132
HOBBIES_1,3323.7006,3204.7920,3229.0146,3236.0135,3687.4460,4288.1278,3967.1454,3328.0964,3209.1879,3233.4104,...,3696.2377,4296.9195,3975.9370,3336.8881,3217.9796,3242.2021,3249.2011,3700.6335,4301.3153,3980.3329
HOBBIES_2,359.5239,359.7182,373.1168,370.4114,385.8900,421.8119,420.6703,360.2707,360.4650,373.8635,...,387.3835,423.3054,422.1639,361.7643,361.9586,375.3571,372.6518,388.1303,424.0522,422.9107
HOUSEHOLD_1,7408.2645,6725.2453,6670.2286,6831.7739,7789.1451,9827.5156,9604.6879,7424.8168,6741.7976,6686.7810,...,7822.2497,9860.6202,9637.7925,7457.9214,6774.9022,6719.8856,6881.4308,7838.8020,9877.1725,9654.3448
HOUSEHOLD_2,1939.1333,1828.7606,1841.0783,1880.7107,2115.5529,2557.1659,2497.9005,1942.6239,1832.2513,1844.5690,...,2122.5343,2564.1472,2504.8818,1949.6052,1839.2326,1851.5503,1891.1827,2126.0249,2567.6379,2508.3724


# Level 4

In [33]:
ts_4 = df_sales_store_dept.reset_index()
ts_4['dept_id'] = ts_4['dept_id'].str[:-2]
ts_4 = ts_4.groupby(['dept_id']).sum()

t_4 = truth.T.reset_index()
t_4['dept_id'] = t_4['dept_id'].str[:-2]
t_4 = t_4.groupby(['dept_id']).sum()

p_4 = df_bu.copy()
p_4['dept_id'] = p_4['dept_id'].str[:-2]
p_4 = p_4.groupby(['dept_id']).sum()

display(ts_4, t_4, p_4)

rmsse_4 = []

for row in t_4.index:
    rmsse_4.append(rmsse(t_4.loc[row].to_numpy(), p_4.loc[row].to_numpy(),
                         ts_4.loc[row].to_numpy()))
    
df_levels[4] = [sum([i*j for i,j in zip(rmsse_4, level_weights[4])])]

,2011-01-29,2011-01-30,2011-01-31,2011-02-01,2011-02-02,2011-02-03,2011-02-04,2011-02-05,2011-02-06,2011-02-07,...,2016-04-15,2016-04-16,2016-04-17,2016-04-18,2016-04-19,2016-04-20,2016-04-21,2016-04-22,2016-04-23,2016-04-24
dept_id,,,,,,,,,,,,,,,,,,,,,
FOODS,23178.0,22758.0,17174.0000,18878.0000,15707.5000,22093.0,20490.0000,27751.0,24862.0000,18901.0000,...,28682.0,32007.0,34497.0,26151.0,24948.0,23632.0,23317.0,26704.0,31927.0,32654.0
HOBBIES,3944.0,3558.2,2814.7333,2827.2667,2251.1333,3368.0,3139.6667,4159.0,3051.8333,2782.6667,...,3786.0,4634.0,4820.0,3323.0,3787.0,3472.0,3353.0,4085.0,4787.0,4683.0
HOUSEHOLD,5689.0,5634.0,3927.0000,3865.0000,2970.0000,3898.0,4576.0000,6195.0,4975.0000,4056.0000,...,9321.0,11721.0,12323.0,8585.0,8835.0,8239.0,8363.0,9728.0,12248.0,12458.0


,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
dept_id,,,,,,,,,,,,,,,,,,,,,
FOODS,25953,23519,22872,22915,27668,32723,35369,29142,29594,27199,...,30107,36576,41447,29158,26148,24790,24737,28136,33599,35967
HOBBIES,3782,3645,3730,3742,4085,5214,4858,4059,4373,3708,...,3871,4893,5016,3528,3629,3740,3475,4143,5333,5280
HOUSEHOLD,9058,8323,7843,8075,11143,12492,12805,9980,10347,8694,...,10095,12839,13458,9676,9000,8566,8751,10273,12586,13091


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
dept_id,,,,,,,,,,,,,,,,,,,,,
FOODS,25804.9686,24170.5145,23869.2458,24107.3700,26664.858,32193.0804,32729.7260,25832.7051,24198.2510,23896.9822,...,26720.3309,32248.5533,32785.1990,25888.1780,24253.7239,23952.4552,24190.5795,26748.0674,32276.2898,32812.9354
HOBBIES,3683.2245,3564.5103,3602.1314,3606.4249,4073.336,4709.9397,4387.8157,3688.3671,3569.6529,3607.2740,...,4083.6212,4720.2249,4398.1010,3698.6524,3579.9382,3617.5592,3621.8528,4088.7638,4725.3676,4403.2436
HOUSEHOLD,9347.3978,8554.0059,8511.3070,8712.4846,9904.698,12384.6815,12102.5884,9367.4407,8574.0489,8531.3499,...,9944.7839,12424.7674,12142.6743,9407.5267,8614.1348,8571.4359,8772.6135,9964.8269,12444.8104,12162.7173


# Level 3

In [34]:
ts_3 = df_sales_store_dept.reset_index()
ts_3 = ts_3.groupby(['store_id']).sum()

t_3 = truth.T.reset_index()
t_3 = t_3.groupby(['store_id']).sum()

p_3 = df_bu.copy()
p_3 = p_3.groupby(['store_id']).sum()

display(ts_3, t_3, p_3)

rmsse_3 = []

for row in t_3.index:
    rmsse_3.append(rmsse(t_3.loc[row].to_numpy(), p_3.loc[row].to_numpy(),
                         ts_3.loc[row].to_numpy()))
    
df_levels[3] = [sum([i*j for i,j in zip(rmsse_3, level_weights[3])])]

,2011-01-29,2011-01-30,2011-01-31,2011-02-01,2011-02-02,2011-02-03,2011-02-04,2011-02-05,2011-02-06,2011-02-07,...,2016-04-15,2016-04-16,2016-04-17,2016-04-18,2016-04-19,2016-04-20,2016-04-21,2016-04-22,2016-04-23,2016-04-24
store_id,,,,,,,,,,,,,,,,,,,,,
CA_1,4337.0,4171.2,2832.4000,3061.6000,2641.8000,3276.0000,3459.5000,5437.0,4349.3333,3161.6667,...,3982.0,5437.0,5954.0,4345.0,3793.0,3722.0,3709.0,4387.0,5577.0,6113.0
CA_2,3494.0,3046.0,2121.0000,2332.5000,1942.0000,2290.5000,2629.0000,3729.0,2957.0000,2236.0000,...,4440.0,5352.0,5760.0,3830.0,3631.0,3691.0,3303.0,4457.0,5884.0,6082.0
CA_3,4739.0,4827.0,3785.0000,4232.0000,3817.0000,4377.5000,4703.0000,5456.0,5581.0000,4912.0000,...,5337.0,6936.0,8271.0,6068.0,5683.0,5235.0,5018.0,5623.0,7419.0,7721.0
CA_4,1805.0,1955.0,1498.0000,1551.0000,1635.0000,1498.0000,1628.0000,2161.0,1943.0000,1653.0000,...,2496.0,2839.0,3047.0,2809.0,2677.0,2500.0,2458.0,2628.0,2954.0,3271.0
TX_1,2556.0,2687.0,1822.0000,2258.0000,1694.0000,2739.6667,1693.3333,2820.0,2887.0000,2182.0000,...,3084.0,3724.0,4192.0,3410.0,3257.0,2901.0,2776.0,3022.0,3700.0,4033.0
TX_2,3852.0,3937.0,2731.0000,2954.0000,2492.0000,3439.0000,2588.0000,3772.0,3657.0000,2932.0000,...,3897.0,4475.0,4998.0,3311.0,3727.0,3384.0,3446.0,3902.0,4483.0,4292.0
TX_3,3030.0,3006.0,2225.0000,2192.5000,1726.0000,2842.3333,1958.6667,2848.0,2832.0000,2220.5000,...,3819.0,4261.0,4519.0,3147.0,3938.0,3315.0,3380.0,3691.0,4083.0,3957.0
WI_1,2704.0,2194.0,1566.3333,1255.6667,1654.0000,2052.3333,2818.6667,3248.0,1679.5000,1355.0000,...,3862.0,4862.0,4812.0,3236.0,3069.0,3242.0,3324.0,3991.0,4772.0,4874.0
WI_2,2256.0,1929.0,2018.0000,2522.0000,1187.3333,2249.6667,2232.0000,2643.0,2140.0000,1847.5000,...,6259.0,5579.0,5566.0,4347.0,4464.0,4194.0,4393.0,4988.0,5404.0,5127.0


,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
store_id,,,,,,,,,,,,,,,,,,,,,
CA_1,4472,3703,3715,3618,4573,5898,6397,4794,4594,4234,...,4951,6245,6707,4568,3949,3995,4136,4433,5764,6289
CA_2,3926,3525,3527,3754,4382,6349,6193,4434,3977,4243,...,4839,6507,6710,4238,3963,3789,4037,4751,7120,6614
CA_3,6359,5289,5065,5015,5705,7057,7828,6776,6325,6204,...,6046,7029,7969,5891,5523,5375,5580,5542,7073,8144
CA_4,2767,2495,2529,2277,2520,3023,3271,3113,2934,2585,...,2635,3256,3318,3024,2715,2519,2544,2704,3146,3597
TX_1,3076,2853,2984,2664,3687,4052,4412,2859,3217,2839,...,3610,3787,5000,3708,3310,3147,2748,3664,4167,4624
TX_2,3883,3502,3256,3441,4023,4508,4872,3831,4088,3612,...,4415,4424,5866,4179,4008,3518,3126,4249,4802,5217
TX_3,3703,3578,3335,3550,4452,4134,4909,4029,4644,3458,...,4295,4155,5383,4341,4052,3710,3288,4390,4712,4974
WI_1,3166,3194,3267,3201,4143,5318,5228,3502,3456,3444,...,3978,5527,5488,3505,3323,3242,3478,3813,5002,5054
WI_2,4178,4148,3805,4342,5719,5485,5348,5597,6567,5175,...,5317,7704,7586,5011,4628,4533,4628,4880,5213,5068


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
store_id,,,,,,,,,,,,,,,,,,,,,
CA_1,4426.8075,3968.2525,3889.6967,3881.1146,4492.5239,5720.4467,5826.2061,4431.0647,3972.5097,3893.9539,...,4501.0383,5728.9611,5834.7205,4439.5791,3981.0241,3902.4683,3893.8862,4505.2955,5733.2183,5838.9777
CA_2,3921.0027,3732.7987,3706.8634,3713.6540,4412.8179,5850.2141,5810.3909,3929.6055,3741.4015,3715.4662,...,4430.0235,5867.4197,5827.5965,3946.8111,3758.6071,3732.6718,3739.4624,4438.6263,5876.0225,5836.1993
CA_3,6176.0123,5670.9575,5469.0271,5436.6329,5787.8378,7174.9271,7603.1014,6182.8822,5677.8274,5475.8970,...,5801.5776,7188.6669,7616.8412,6196.6220,5691.5673,5489.6368,5457.2427,5808.4475,7195.5369,7623.7112
CA_4,2763.8346,2559.8043,2500.6305,2524.3613,2616.5168,2983.0281,3056.5908,2768.2659,2564.2356,2505.0618,...,2625.3794,2991.8907,3065.4534,2777.1285,2573.0982,2513.9244,2537.6552,2629.8107,2996.3220,3069.8847
TX_1,3109.2603,2882.0448,2864.2225,2884.3919,3110.9815,3776.3874,4014.0387,3113.1434,2885.9280,2868.1057,...,3118.7478,3784.1538,4021.8050,3120.9098,2893.6944,2875.8721,2896.0414,3122.6310,3788.0370,4025.6882
TX_2,3695.9890,3315.0993,3334.5813,3433.5730,3741.2215,4449.6830,4642.1692,3698.1402,3317.2505,3336.7325,...,3745.5239,4453.9853,4646.4716,3702.4425,3321.5528,3341.0348,3440.0265,3747.6751,4456.1365,4648.6227
TX_3,3573.0743,3275.5470,3208.3897,3261.4179,3519.4479,4110.6397,4210.7903,3577.2704,3279.7431,3212.5858,...,3527.8401,4119.0319,4219.1826,3585.6626,3288.1353,3220.9780,3274.0062,3532.0362,4123.2280,4223.3787
WI_1,3162.1533,3012.7672,3141.6313,3221.5099,4010.7548,5191.0219,4684.0798,3169.1776,3019.7916,3148.6556,...,4024.8035,5205.0706,4698.1285,3183.2263,3033.8402,3162.7043,3242.5829,4031.8278,5212.0949,4705.1528
WI_2,4575.3281,4498.0675,4575.5887,4668.9602,5031.0695,5449.9251,5082.8484,4585.6734,4508.4129,4585.9341,...,5051.7602,5470.6158,5103.5391,4606.3642,4529.1036,4606.6248,4699.9963,5062.1055,5480.9612,5113.8845


# Level 2

In [35]:
ts_2 = df_sales_store_dept.reset_index()
ts_2['store_id'] = ts_2['store_id'].str[:-2]
ts_2 = ts_2.groupby(['store_id']).sum()

t_2 = truth.T.reset_index()
t_2['store_id'] = t_2['store_id'].str[:-2]
t_2 = t_2.groupby(['store_id']).sum()

p_2 = df_bu.copy()
p_2['store_id'] = p_2['store_id'].str[:-2]
p_2 = p_2.groupby(['store_id']).sum()

display(ts_2, t_2, p_2)

rmsse_2 = []

for row in t_2.index:
    rmsse_2.append(rmsse(t_2.loc[row].to_numpy(), p_2.loc[row].to_numpy(),
                         ts_2.loc[row].to_numpy()))
    
df_levels[2] = [sum([i*j for i,j in zip(rmsse_2, level_weights[2])])]

,2011-01-29,2011-01-30,2011-01-31,2011-02-01,2011-02-02,2011-02-03,2011-02-04,2011-02-05,2011-02-06,2011-02-07,...,2016-04-15,2016-04-16,2016-04-17,2016-04-18,2016-04-19,2016-04-20,2016-04-21,2016-04-22,2016-04-23,2016-04-24
store_id,,,,,,,,,,,,,,,,,,,,,
CA,14375.0,13999.2,10236.4000,11177.1000,10035.8000,11442.0,12419.5000,16783.0,14830.3333,11962.6667,...,16255.0,20564.0,23032.0,17052.0,15784.0,15148.0,14488.0,17095.0,21834.0,23187.0
TX,9438.0,9630.0,6778.0000,7404.5000,5912.0000,9021.0,6240.0000,9440.0,9376.0000,7334.5000,...,10800.0,12460.0,13709.0,9868.0,10922.0,9600.0,9602.0,10615.0,12266.0,12282.0
WI,8998.0,8321.0,6901.3333,6988.6667,4980.8333,8896.0,9546.1667,11882.0,8682.5000,6442.5000,...,14734.0,15338.0,14899.0,11139.0,10864.0,10595.0,10943.0,12807.0,14862.0,14326.0


,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
store_id,,,,,,,,,,,,,,,,,,,,,
CA,17524,15012,14836,14664,17180,22327,23689,19117,17830,17266,...,18471,23037,24704,17721,16150,15678,16297,17430,23103,24644
TX,10662,9933,9575,9655,12162,12694,14193,10719,11949,9909,...,12320,12366,16249,12228,11370,10375,9162,12303,13681,14815
WI,10607,10542,10034,10413,13554,15408,15150,13345,14535,12426,...,13282,18905,18968,12413,11257,11043,11504,12819,14734,14879


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
store_id,,,,,,,,,,,,,,,,,,,,,
CA,17287.6570,15931.8129,15566.2176,15555.7629,17309.6963,21728.6161,22296.2892,17311.8182,15955.9742,15590.3789,...,17358.0188,21776.9385,22344.6117,17360.1407,16004.2967,15638.7014,15628.2466,17382.1800,21801.0998,22368.7729
TX,10378.3236,9472.6911,9407.1935,9579.3828,10371.6509,12336.7101,12866.9983,10388.5541,9482.9216,9417.4239,...,10392.1118,12357.1710,12887.4592,10409.0150,9503.3825,9437.8849,9610.0741,10402.3423,12367.4015,12897.6896
WI,11169.6103,10884.5267,11009.2730,11291.1339,12961.5447,15222.3754,14056.8426,11188.1406,10903.0570,11027.8033,...,12998.6055,15259.4362,14093.9034,11225.2014,10940.1178,11064.8641,11346.7250,13017.1359,15277.9666,14112.4337


# Level 1

In [36]:
ts_1 = df_sales_store_dept.sum(0)

t_1 = truth.T.sum(0)

p_1 = df_bu.iloc[:, 2:].sum(0)

display(ts_1, t_1, p_1)

rmsse_1 = [rmsse(t_1.to_numpy(), p_1.to_numpy(),
                         ts_1.to_numpy())]
    
df_levels[1] = [sum([i*j for i,j in zip(rmsse_1, level_weights[1])])]

2011-01-29    32811.0000
2011-01-30    31950.2000
2011-01-31    23915.7333
2011-02-01    25570.2667
2011-02-02    20928.6333
                 ...    
2016-04-20    35343.0000
2016-04-21    35033.0000
2016-04-22    40517.0000
2016-04-23    48962.0000
2016-04-24    49795.0000
Freq: D, Length: 1913, dtype: float64

d_1914    38793
d_1915    35487
d_1916    34445
d_1917    34732
d_1918    42896
d_1919    50429
d_1920    53032
d_1921    43181
d_1922    44314
d_1923    39601
d_1924    40763
d_1925    43805
d_1926    54239
d_1927    45609
d_1928    46400
d_1929    39379
d_1930    42248
d_1931    40503
d_1932    44073
d_1933    54308
d_1934    59921
d_1935    42362
d_1936    38777
d_1937    37096
d_1938    36963
d_1939    42552
d_1940    51518
d_1941    54338
dtype: int64

0     38835.5909
1     36289.0307
2     35982.6841
3     36426.2795
4     40642.8920
5     49287.7016
6     49220.1301
7     38888.5129
8     36341.9528
9     36035.6061
10    36479.2016
11    40695.8140
12    49340.6237
13    49273.0521
14    38941.4350
15    36394.8748
16    36088.5282
17    36532.1237
18    40748.7361
19    49393.5457
20    49325.9742
21    38994.3571
22    36447.7969
23    36141.4503
24    36585.0458
25    40801.6582
26    49446.4678
27    49378.8963
dtype: float64

In [37]:
df_levels.index = ['AHW-WRMSSE']
df_levels

,9,8,7,6,5,4,3,2,1
AHW-WRMSSE,0.8442,0.8082,0.6393,0.7862,0.8442,0.7875,0.7994,0.7707,0.7782


### Q7. Top-Down  (15 pts.)

Using your forecasts from the best performing method in Q5, use the top-down method with `average historical proportions` described in [FPP3](https://otexts.com/fpp3/single-level.html) to generate forecasts for levels 10 to 12.

Calculate the `WRMSSE` for levels 10 to 12  against the test set, then summarize the metrics in a dataframe and print it.

For reference, you can find the benchmark `WRMSSE` scores in the `The M5 Accuracy competition: Results, findings and conclusions` paper.

<i>Note: The M5 benchmarks use a bottom-up method for forecasting, so they will not necessarily be equal to your scores.</i>

In [38]:
df_td = df_bu.copy()
ts_td = df_sales_2.copy()

# Level 10

In [39]:
df_10 = df_td.copy()

ts_10 = ts_td.groupby('item_id').sum().iloc[:, :-28]

t_10 = ts_td.groupby('item_id').sum().iloc[:, -28:]

r_10 = ts_td.iloc[:, :-28].groupby(['dept_id', 'item_id'])['item_id'].count()
r_10 = r_10.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
df_10['item_id'] = df_10.dept_id.apply(lambda x: ts_td[ts_td.dept_id==x].item_id.unique())
p_10 = df_10.explode('item_id').groupby('item_id').sum().multiply(r_10.droplevel(level=0), axis=0)/100

display(ts_10, t_10, p_10, r_10)

rmsse_10 = []

for row in t_10.index:
    rmsse_10.append(rmsse(t_10.loc[row].to_numpy(), p_10.loc[row].to_numpy(),
                         ts_10.loc[row].to_numpy()))
    
df_levels[10] = [sum([i*j for i,j in zip(rmsse_10, level_weights[10])])]

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
item_id,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001,6,6,4,6,7,18,10,4,11,10,...,4,4,30,7,5,3,6,2,16,6
FOODS_1_002,4,5,7,4,3,4,1,7,2,4,...,5,9,4,1,3,5,5,3,3,1
FOODS_1_003,14,8,3,6,3,8,13,10,11,6,...,7,3,5,6,3,4,4,3,11,5
FOODS_1_004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
FOODS_1_005,34,32,13,20,10,21,18,20,25,41,...,16,14,14,18,18,27,12,15,38,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_512,5,4,1,3,2,4,2,8,7,5,...,6,7,9,13,12,2,8,6,10,5
HOUSEHOLD_2_513,0,0,0,0,0,0,0,0,0,0,...,9,3,3,3,2,4,6,7,4,11
HOUSEHOLD_2_514,4,8,2,1,1,2,3,8,2,1,...,1,2,2,0,1,0,0,2,2,2


,d_1914,d_1915,d_1916,d_1917,d_1918,d_1919,d_1920,d_1921,d_1922,d_1923,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
item_id,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001,4,5,7,1,9,3,9,1,10,3,...,7,8,8,4,7,7,5,7,5,9
FOODS_1_002,9,4,3,4,5,7,2,5,6,3,...,5,8,7,2,6,5,0,6,6,4
FOODS_1_003,7,13,9,1,2,4,7,9,2,3,...,7,8,6,11,9,7,7,10,6,5
FOODS_1_004,0,15,33,67,118,101,132,119,156,116,...,115,104,107,81,73,62,71,75,83,93
FOODS_1_005,14,9,27,14,14,33,19,17,11,16,...,15,16,25,9,7,25,19,25,17,19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_512,6,3,5,5,12,3,8,9,5,5,...,5,7,11,4,8,3,4,5,4,7
HOUSEHOLD_2_513,2,4,0,4,2,5,3,2,5,2,...,4,0,5,3,6,3,0,9,3,4
HOUSEHOLD_2_514,0,2,0,0,2,2,4,1,3,0,...,2,2,2,6,1,0,1,4,3,3


,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
item_id,,,,,,,,,,,,,,,,,,,,,
FOODS_1_001,12.0805,12.4367,12.5789,13.1121,14.7554,16.6184,14.7325,12.0986,12.4547,12.5970,...,14.7916,16.6545,14.7687,12.1347,12.4908,12.6331,13.1663,14.8096,16.6726,14.7867
FOODS_1_002,12.0805,12.4367,12.5789,13.1121,14.7554,16.6184,14.7325,12.0986,12.4547,12.5970,...,14.7916,16.6545,14.7687,12.1347,12.4908,12.6331,13.1663,14.8096,16.6726,14.7867
FOODS_1_003,12.0805,12.4367,12.5789,13.1121,14.7554,16.6184,14.7325,12.0986,12.4547,12.5970,...,14.7916,16.6545,14.7687,12.1347,12.4908,12.6331,13.1663,14.8096,16.6726,14.7867
FOODS_1_004,12.0805,12.4367,12.5789,13.1121,14.7554,16.6184,14.7325,12.0986,12.4547,12.5970,...,14.7916,16.6545,14.7687,12.1347,12.4908,12.6331,13.1663,14.8096,16.6726,14.7867
FOODS_1_005,12.0805,12.4367,12.5789,13.1121,14.7554,16.6184,14.7325,12.0986,12.4547,12.5970,...,14.7916,16.6545,14.7687,12.1347,12.4908,12.6331,13.1663,14.8096,16.6726,14.7867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HOUSEHOLD_2_512,3.7653,3.5510,3.5749,3.6519,4.1079,4.9654,4.8503,3.7721,3.5578,3.5817,...,4.1214,4.9789,4.8638,3.7856,3.5713,3.5952,3.6722,4.1282,4.9857,4.8706
HOUSEHOLD_2_513,3.7653,3.5510,3.5749,3.6519,4.1079,4.9654,4.8503,3.7721,3.5578,3.5817,...,4.1214,4.9789,4.8638,3.7856,3.5713,3.5952,3.6722,4.1282,4.9857,4.8706
HOUSEHOLD_2_514,3.7653,3.5510,3.5749,3.6519,4.1079,4.9654,4.8503,3.7721,3.5578,3.5817,...,4.1214,4.9789,4.8638,3.7856,3.5713,3.5952,3.6722,4.1282,4.9857,4.8706


dept_id      item_id        
FOODS_1      FOODS_1_001        0.4630
             FOODS_1_002        0.4630
             FOODS_1_003        0.4630
             FOODS_1_004        0.4630
             FOODS_1_005        0.4630
                                 ...  
HOUSEHOLD_2  HOUSEHOLD_2_512    0.1942
             HOUSEHOLD_2_513    0.1942
             HOUSEHOLD_2_514    0.1942
             HOUSEHOLD_2_515    0.1942
             HOUSEHOLD_2_516    0.1942
Name: item_id, Length: 3049, dtype: float64

# Level 11

In [40]:
df_11 = df_td.copy()

ts_11 = ts_td.groupby(['state_id', 'item_id']).sum().iloc[:, :-28]

t_11 = ts_td.groupby(['state_id', 'item_id']).sum().iloc[:, -28:]

r_11 = ts_td.iloc[:, :-28].groupby(['state_id', 'item_id'])['id'].count()
r_11 = r_11.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
df_11['item_id'] = df_11.apply(lambda x: ts_td.loc[(ts_td['dept_id']==x['dept_id']) & 
                                                   (ts_td['store_id']==x['store_id']), 'item_id'].unique(), axis=1)
p_11 = df_11.explode('item_id')
p_11['store_id'] = p_11['store_id'].str[:-2]
p_11 = p_11.groupby(['store_id', 'item_id']).sum().multiply(r_11, axis=0)/100

display(ts_11, t_11, p_11, r_11)

rmsse_11 = []

for row in t_11.index:
    rmsse_11.append(rmsse(t_11.loc[row].to_numpy(), p_11.loc[row].to_numpy(),
                         ts_11.loc[row].to_numpy()))
    
df_levels[11] = [sum([i*j for i,j in zip(rmsse_11, level_weights[11])])]

d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  \
state_id item_id                                                              
CA       FOODS_1_001        6    3    2    3    7    5    8    3    5     2   
         FOODS_1_002        3    3    4    4    3    3    0    2    1     1   
         FOODS_1_003        9    4    3    4    2    5    7    3    4     2   
         FOODS_1_004        0    0    0    0    0    0    0    0    0     0   
         FOODS_1_005       23   13    6   15    7    8    7    9   19    30   
...                       ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
WI       HOUSEHOLD_2_512    2    3    0    1    0    0    0    4    2     1   
         HOUSEHOLD_2_513    0    0    0    0    0    0    0    0    0     0   
         HOUSEHOLD_2_514    1    1    1    0    0    0    0    3    0     0   
         HOUSEHOLD_2_515    0    0    0    0    0    0    0    0    0     0   
         HOUSEHOLD_2_516    0    1    0    2    0    1    1    1    0     0   

                          ...  d_1904  d_1905  d_1906  d_1907  d_1908  d_1909  \
state_id item_id          ...                                                   
CA       FOODS_1_001      ...       1       2      27       6       2       2   
         FOODS_1_002      ...       3       5       2       0       0       2   
         FOODS_1_003      ...       7       1       5       5       3       3   
         FOODS_1_004      ...       0       0       0       0       0       0   
         FOODS_1_005      ...      10       8       4       4       8      10   
...                       ...     ...     ...     ...     ...     ...     ...   
WI       HOUSEHOLD_2_512  ...       0       2       2       6       2       1   
         HOUSEHOLD_2_513  ...       0       0       1       1       1       1   
         HOUSEHOLD_2_514  ...       0       0       0       0       0       0   
         HOUSEHOLD_2_515  ...       0       1       0       0       0       1   
         HOUSEHOLD_2_516  ...       1       0       0       0       0       0   

                          d_1910  d_1911  d_1912  d_1913  
state_id item_id                                          
CA       FOODS_1_001           4       2       3       5  
         FOODS_1_002           2       1       2       1  
         FOODS_1_003           3       1       7       3  
         FOODS_1_004           0       0       0       0  
         FOODS_1_005           3       9      25       3  
...                          ...     ...     ...     ...  
WI       HOUSEHOLD_2_512       1       2       4       0  
         HOUSEHOLD_2_513       1       2       0       0  
         HOUSEHOLD_2_514       0       1       1       0  
         HOUSEHOLD_2_515       1       1       1       0  
         HOUSEHOLD_2_516       1       0       0       0  

[9147 rows x 1913 columns]

d_1914  d_1915  d_1916  d_1917  d_1918  d_1919  \
state_id item_id                                                           
CA       FOODS_1_001           3       4       1       0       8       3   
         FOODS_1_002           1       2       2       3       3       4   
         FOODS_1_003           6       2       3       1       0       1   
         FOODS_1_004           0       9      11       8      12       7   
         FOODS_1_005           8       4      16       7       6      11   
...                          ...     ...     ...     ...     ...     ...   
WI       HOUSEHOLD_2_512       4       0       1       4       3       1   
         HOUSEHOLD_2_513       0       0       0       0       1       1   
         HOUSEHOLD_2_514       0       0       0       0       0       0   
         HOUSEHOLD_2_515       1       1       0       0       0       0   
         HOUSEHOLD_2_516       1       0       0       1       1       0   

                          d_1920  d_1921  d_1922  d_1923  ...  d_1932  d_1933  \
state_id item_id                                          ...                   
CA       FOODS_1_001           6       0       7       1  ...       5       5   
         FOODS_1_002           0       1       5       1  ...       3       4   
         FOODS_1_003           3       3       1       3  ...       4       3   
         FOODS_1_004          16      11      24      25  ...      30      35   
         FOODS_1_005          10       9       4       8  ...      10       8   
...                          ...     ...     ...     ...  ...     ...     ...   
WI       HOUSEHOLD_2_512       1       1       0       0  ...       3       1   
         HOUSEHOLD_2_513       0       0       1       1  ...       1       0   
         HOUSEHOLD_2_514       0       1       1       0  ...       0       0   
         HOUSEHOLD_2_515       0       0       0       0  ...       0       0   
         HOUSEHOLD_2_516       0       0       0       0  ...       0       1   

                          d_1934  d_1935  d_1936  d_1937  d_1938  d_1939  \
state_id item_id                                                           
CA       FOODS_1_001           3       1       1       2       2       4   
         FOODS_1_002           6       2       1       2       0       4   
         FOODS_1_003           4       8       8       5       3       3   
         FOODS_1_004          15      20      15      24      13      17   
         FOODS_1_005          13       4       2       5      11      21   
...                          ...     ...     ...     ...     ...     ...   
WI       HOUSEHOLD_2_512       4       3       0       2       0       0   
         HOUSEHOLD_2_513       0       0       0       0       0       0   
         HOUSEHOLD_2_514       0       1       0       0       0       3   
         HOUSEHOLD_2_515       1       4       1       1       0       0   
         HOUSEHOLD_2_516       1       1       0       0       0       1   

                          d_1940  d_1941  
state_id item_id                          
CA       FOODS_1_001           4       2  
         FOODS_1_002           4       4  
         FOODS_1_003           4       4  
         FOODS_1_004          26      37  
         FOODS_1_005           7      14  
...                          ...     ...  
WI       HOUSEHOLD_2_512       0       3  
         HOUSEHOLD_2_513       0       0  
         HOUSEHOLD_2_514       0       0  
         HOUSEHOLD_2_515       0       1  
         HOUSEHOLD_2_516       0       0  

[9147 rows x 28 columns]

0       1       2       3      4       5  \
store_id item_id                                                          
CA       FOODS_1_001      0.3969  0.3951  0.3972  0.4172  0.475  0.5559   
         FOODS_1_002      0.3969  0.3951  0.3972  0.4172  0.475  0.5559   
         FOODS_1_003      0.3969  0.3951  0.3972  0.4172  0.475  0.5559   
         FOODS_1_004      0.3969  0.3951  0.3972  0.4172  0.475  0.5559   
         FOODS_1_005      0.3969  0.3951  0.3972  0.4172  0.475  0.5559   
...                          ...     ...     ...     ...    ...     ...   
WI       HOUSEHOLD_2_512  0.1447  0.1395  0.1415  0.1480  0.174  0.1981   
         HOUSEHOLD_2_513  0.1447  0.1395  0.1415  0.1480  0.174  0.1981   
         HOUSEHOLD_2_514  0.1447  0.1395  0.1415  0.1480  0.174  0.1981   
         HOUSEHOLD_2_515  0.1447  0.1395  0.1415  0.1480  0.174  0.1981   
         HOUSEHOLD_2_516  0.1447  0.1395  0.1415  0.1480  0.174  0.1981   

                               6       7       8       9  ...      18      19  \
store_id item_id                                          ...                   
CA       FOODS_1_001      0.5009  0.3974  0.3957  0.3978  ...  0.4761  0.5570   
         FOODS_1_002      0.5009  0.3974  0.3957  0.3978  ...  0.4761  0.5570   
         FOODS_1_003      0.5009  0.3974  0.3957  0.3978  ...  0.4761  0.5570   
         FOODS_1_004      0.5009  0.3974  0.3957  0.3978  ...  0.4761  0.5570   
         FOODS_1_005      0.5009  0.3974  0.3957  0.3978  ...  0.4761  0.5570   
...                          ...     ...     ...     ...  ...     ...     ...   
WI       HOUSEHOLD_2_512  0.1820  0.1450  0.1398  0.1418  ...  0.1745  0.1987   
         HOUSEHOLD_2_513  0.1820  0.1450  0.1398  0.1418  ...  0.1745  0.1987   
         HOUSEHOLD_2_514  0.1820  0.1450  0.1398  0.1418  ...  0.1745  0.1987   
         HOUSEHOLD_2_515  0.1820  0.1450  0.1398  0.1418  ...  0.1745  0.1987   
         HOUSEHOLD_2_516  0.1820  0.1450  0.1398  0.1418  ...  0.1745  0.1987   

                              20      21      22      23      24      25  \
store_id item_id                                                           
CA       FOODS_1_001      0.5020  0.3985  0.3968  0.3989  0.4189  0.4767   
         FOODS_1_002      0.5020  0.3985  0.3968  0.3989  0.4189  0.4767   
         FOODS_1_003      0.5020  0.3985  0.3968  0.3989  0.4189  0.4767   
         FOODS_1_004      0.5020  0.3985  0.3968  0.3989  0.4189  0.4767   
         FOODS_1_005      0.5020  0.3985  0.3968  0.3989  0.4189  0.4767   
...                          ...     ...     ...     ...     ...     ...   
WI       HOUSEHOLD_2_512  0.1826  0.1456  0.1403  0.1424  0.1489  0.1748   
         HOUSEHOLD_2_513  0.1826  0.1456  0.1403  0.1424  0.1489  0.1748   
         HOUSEHOLD_2_514  0.1826  0.1456  0.1403  0.1424  0.1489  0.1748   
         HOUSEHOLD_2_515  0.1826  0.1456  0.1403  0.1424  0.1489  0.1748   
         HOUSEHOLD_2_516  0.1826  0.1456  0.1403  0.1424  0.1489  0.1748   

                              26      27  
store_id item_id                          
CA       FOODS_1_001      0.5575  0.5025  
         FOODS_1_002      0.5575  0.5025  
         FOODS_1_003      0.5575  0.5025  
         FOODS_1_004      0.5575  0.5025  
         FOODS_1_005      0.5575  0.5025  
...                          ...     ...  
WI       HOUSEHOLD_2_512  0.1989  0.1829  
         HOUSEHOLD_2_513  0.1989  0.1829  
         HOUSEHOLD_2_514  0.1989  0.1829  
         HOUSEHOLD_2_515  0.1989  0.1829  
         HOUSEHOLD_2_516  0.1989  0.1829  

[9147 rows x 28 columns]

state_id  item_id        
CA        FOODS_1_001        0.0328
          FOODS_1_002        0.0328
          FOODS_1_003        0.0328
          FOODS_1_004        0.0328
          FOODS_1_005        0.0328
                              ...  
WI        HOUSEHOLD_2_512    0.0328
          HOUSEHOLD_2_513    0.0328
          HOUSEHOLD_2_514    0.0328
          HOUSEHOLD_2_515    0.0328
          HOUSEHOLD_2_516    0.0328
Name: id, Length: 9147, dtype: float64

# Level 12

In [41]:
df_12 = df_td.copy()

ts_12 = ts_td.groupby(['item_id', 'store_id']).sum().iloc[:, :-28]

t_12 = ts_td.groupby(['item_id', 'store_id']).sum().iloc[:, -28:]

r_12 = ts_td.iloc[:, :-28].groupby([ 'item_id', 'store_id'])['id'].count()
r_12 = r_12.groupby(level=0).apply(lambda x: 100 * x / float(x.sum()))
df_12['item_id'] = df_12.apply(lambda x: ts_td.loc[(ts_td['dept_id']==x['dept_id']) & 
                                                   (ts_td['store_id']==x['store_id']), 'item_id'].unique(), axis=1)
p_12 = df_12.explode('item_id')
p_12 = p_12.groupby([ 'item_id', 'store_id']).sum().multiply(r_12, axis=0)/100

display(ts_12, t_12, p_12, r_12)

rmsse_12 = []

for row in t_12.index:
    rmsse_12.append(rmsse(t_12.loc[row].to_numpy(), p_12.loc[row].to_numpy(),
                         ts_12.loc[row].to_numpy()))
    
df_levels[12] = [sum([i*j for i,j in zip(rmsse_12, level_weights[12])])]

d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  \
item_id         store_id                                                      
FOODS_1_001     CA_1        3    0    0    1    4    2    0    2    0     0   
                CA_2        2    0    0    0    1    0    8    0    4     1   
                CA_3        1    2    1    1    1    2    0    1    1     1   
                CA_4        0    1    1    1    1    1    0    0    0     0   
                TX_1        0    1    1    0    0    0    0    0    2     1   
...                       ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
HOUSEHOLD_2_516 TX_2        0    1    0    0    0    0    1    1    0     0   
                TX_3        2    0    0    0    0    0    0    0    0     0   
                WI_1        0    0    0    1    0    0    0    0    0     0   
                WI_2        0    0    0    0    0    0    0    0    0     0   
                WI_3        0    1    0    1    0    1    1    1    0     0   

                          ...  d_1904  d_1905  d_1906  d_1907  d_1908  d_1909  \
item_id         store_id  ...                                                   
FOODS_1_001     CA_1      ...       0       2       0       4       1       1   
                CA_2      ...       1       0      14       0       1       1   
                CA_3      ...       0       0      13       0       0       0   
                CA_4      ...       0       0       0       2       0       0   
                TX_1      ...       0       0       0       0       0       0   
...                       ...     ...     ...     ...     ...     ...     ...   
HOUSEHOLD_2_516 TX_2      ...       0       1       1       0       0       1   
                TX_3      ...       0       0       0       0       0       0   
                WI_1      ...       1       0       0       0       0       0   
                WI_2      ...       0       0       0       0       0       0   
                WI_3      ...       0       0       0       0       0       0   

                          d_1910  d_1911  d_1912  d_1913  
item_id         store_id                                  
FOODS_1_001     CA_1           0       1       1       0  
                CA_2           4       0       0       4  
                CA_3           0       0       1       0  
                CA_4           0       1       1       1  
                TX_1           0       0       0       0  
...                          ...     ...     ...     ...  
HOUSEHOLD_2_516 TX_2           0       0       0       0  
                TX_3           0       0       0       0  
                WI_1           0       0       0       0  
                WI_2           1       0       0       0  
                WI_3           0       0       0       0  

[30490 rows x 1913 columns]

d_1914  d_1915  d_1916  d_1917  d_1918  d_1919  \
item_id         store_id                                                   
FOODS_1_001     CA_1           2       0       0       0       0       1   
                CA_2           0       3       0       0       0       1   
                CA_3           1       0       1       0       8       1   
                CA_4           0       1       0       0       0       0   
                TX_1           0       0       1       0       1       0   
...                          ...     ...     ...     ...     ...     ...   
HOUSEHOLD_2_516 TX_2           0       0       0       0       0       0   
                TX_3           0       0       0       0       0       0   
                WI_1           1       0       0       1       0       0   
                WI_2           0       0       0       0       0       0   
                WI_3           0       0       0       0       1       0   

                          d_1920  d_1921  d_1922  d_1923  ...  d_1932  d_1933  \
item_id         store_id                                  ...                   
FOODS_1_001     CA_1           1       0       6       1  ...       2       3   
                CA_2           4       0       0       0  ...       1       0   
                CA_3           0       0       1       0  ...       1       2   
                CA_4           1       0       0       0  ...       1       0   
                TX_1           1       1       1       1  ...       1       1   
...                          ...     ...     ...     ...  ...     ...     ...   
HOUSEHOLD_2_516 TX_2           0       1       0       0  ...       0       0   
                TX_3           1       0       0       0  ...       0       1   
                WI_1           0       0       0       0  ...       0       0   
                WI_2           0       0       0       0  ...       0       0   
                WI_3           0       0       0       0  ...       0       1   

                          d_1934  d_1935  d_1936  d_1937  d_1938  d_1939  \
item_id         store_id                                                   
FOODS_1_001     CA_1           1       0       0       0       1       0   
                CA_2           0       1       1       0       0       1   
                CA_3           2       0       0       1       0       3   
                CA_4           0       0       0       1       1       0   
                TX_1           1       1       5       0       2       2   
...                          ...     ...     ...     ...     ...     ...   
HOUSEHOLD_2_516 TX_2           0       1       0       0       0       0   
                TX_3           0       0       0       0       0       1   
                WI_1           0       0       0       0       0       1   
                WI_2           1       1       0       0       0       0   
                WI_3           0       0       0       0       0       0   

                          d_1940  d_1941  
item_id         store_id                  
FOODS_1_001     CA_1           0       0  
                CA_2           2       0  
                CA_3           2       2  
                CA_4           0       0  
                TX_1           0       2  
...                          ...     ...  
HOUSEHOLD_2_516 TX_2           0       1  
                TX_3           1       2  
                WI_1           0       0  
                WI_2           0       0  
                WI_3           0       0  

[30490 rows x 28 columns]

0        1        2        3        4  \
item_id         store_id                                                
FOODS_1_001     CA_1      27.1909  26.7018  26.8343  28.0744  33.6404   
                CA_2      40.0500  40.1582  40.0333  42.7585  50.4468   
                CA_3      32.3550  32.7031  32.6634  34.6278  36.7112   
                CA_4      21.4159  20.9181  21.5890  21.7546  24.0408   
                TX_1      17.4873  17.6501  18.5406  18.9404  20.3675   
...                           ...      ...      ...      ...      ...   
HOUSEHOLD_2_516 TX_2      19.4337  18.5358  18.5294  19.3298  21.2412   
                TX_3      16.1183  15.1971  15.5048  15.6584  17.0667   
                WI_1      15.1220  14.2651  14.7724  15.3172  18.9891   
                WI_2      13.6186  13.3696  13.5702  14.4560  16.1518   
                WI_3      15.3827  14.8948  14.8080  15.3592  17.8989   

                                5        6        7        8        9  ...  \
item_id         store_id                                               ...   
FOODS_1_001     CA_1      39.2104  34.2421  27.2203  26.7312  26.8637  ...   
                CA_2      59.0437  53.0625  40.1105  40.2187  40.0938  ...   
                CA_3      46.0365  42.3306  32.3846  32.7327  32.6930  ...   
                CA_4      25.2038  23.0972  21.4612  20.9634  21.6344  ...   
                TX_1      22.1706  21.8073  17.5101  17.6728  18.5633  ...   
...                           ...      ...      ...      ...      ...  ...   
HOUSEHOLD_2_516 TX_2      24.8982  24.6114  19.4634  18.5655  18.5591  ...   
                TX_3      19.9131  19.3881  16.1466  15.2254  15.5331  ...   
                WI_1      23.1155  20.7494  15.1509  14.2940  14.8013  ...   
                WI_2      17.7639  16.2336  13.6452  13.3961  13.5967  ...   
                WI_3      19.5240  18.5184  15.4126  14.9247  14.8379  ...   

                               18       19       20       21       22  \
item_id         store_id                                                
FOODS_1_001     CA_1      33.6992  39.2692  34.3009  27.2791  26.7900   
                CA_2      50.5678  59.1647  53.1835  40.2315  40.3397   
                CA_3      36.7704  46.0958  42.3898  32.4438  32.7919   
                CA_4      24.1314  25.2944  23.1878  21.5518  21.0540   
                TX_1      20.4130  22.2161  21.8528  17.5555  17.7183   
...                           ...      ...      ...      ...      ...   
HOUSEHOLD_2_516 TX_2      21.3005  24.9576  24.6708  19.5228  18.6249   
                TX_3      17.1233  19.9697  19.4447  16.2033  15.2820   
                WI_1      19.0470  23.1733  20.8072  15.2087  14.3518   
                WI_2      16.2048  17.8169  16.2866  13.6982  13.4491   
                WI_3      17.9587  19.5838  18.5782  15.4725  14.9845   

                               23       24       25       26       27  
item_id         store_id                                               
FOODS_1_001     CA_1      26.9225  28.1626  33.7286  39.2986  34.3302  
                CA_2      40.2148  42.9400  50.6283  59.2252  53.2440  
                CA_3      32.7522  34.7166  36.8000  46.1254  42.4194  
                CA_4      21.7250  21.8905  24.1767  25.3397  23.2331  
                TX_1      18.6088  19.0086  20.4357  22.2388  21.8755  
...                           ...      ...      ...      ...      ...  
HOUSEHOLD_2_516 TX_2      18.6185  19.4189  21.3302  24.9873  24.7005  
                TX_3      15.5897  15.7433  17.1516  19.9980  19.4730  
                WI_1      14.8591  15.4039  19.0759  23.2022  20.8361  
                WI_2      13.6497  14.5355  16.2314  17.8434  16.3131  
                WI_3      14.8977  15.4489  17.9886  19.6137  18.6081  

[30490 rows x 28 columns]

item_id          store_id
FOODS_1_001      CA_1        10.0
                 CA_2        10.0
                 CA_3        10.0
                 CA_4        10.0
                 TX_1        10.0
                             ... 
HOUSEHOLD_2_516  TX_2        10.0
                 TX_3        10.0
                 WI_1        10.0
                 WI_2        10.0
                 WI_3        10.0
Name: id, Length: 30490, dtype: float64

# All Levels WRMSSE

In [42]:
df_levels.reindex(sorted(df_levels.columns), axis=1)

,1,2,3,4,5,6,7,8,9,10,11,12
AHW-WRMSSE,0.7782,0.7707,0.7994,0.7875,0.8442,0.7862,0.8277,0.8082,0.8442,2.3586,2.0253,54.4477
